<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [21]</a>'.</span>

In [1]:
# Parameters
run_date = "2026-01-27"
output_dir = "C:\\Users\\sergi\\OneDrive\\\u0420\u0430\u0431\u043e\u0447\u0438\u0439 \u0441\u0442\u043e\u043b\\ORION_MAIN\\OriON\\signals"


In [2]:
# Parameters
run_date = "2026-01-01"  # papermill replacement
import os
output_dir = os.environ.get("ORION_SIGNALS_DIR", "../signals")
config_path = os.environ.get("DATUM_API_CONFIG_PATH", "../ops/datum_api_config.json")
dry_run = False

# ensure output exists
os.makedirs(output_dir, exist_ok=True)


In [3]:
# Import basic modules
import pandas as pd
from datum_api_client import DatumApi
import datetime
from datetime import timedelta

# Import warnings
import warnings
warnings.filterwarnings("ignore")
# pip install xlrd
# pip install openpyxl

# Download excel with ticker




In [4]:
import os
from pathlib import Path

def find_orion_home() -> Path:
    # 1) найнадійніше — env var
    env = os.environ.get("ORION_HOME")
    if env:
        p = Path(env).expanduser().resolve()
        if not p.exists():
            raise FileNotFoundError(f"ORION_HOME points to missing path: {p}")
        return p

    # 2) fallback: шукаємо папку OriON вгору від поточної директорії
    here = Path.cwd().resolve()
    for parent in [here] + list(here.parents):
        if parent.name.lower() == "orion":
            return parent
        cand = parent / "OriON"
        if cand.exists() and cand.is_dir():
            return cand.resolve()

    raise RuntimeError("Cannot locate OriON. Set ORION_HOME env var.")

ORION_HOME = find_orion_home()

CRACEN_DIR = ORION_HOME / "CRACEN"
WORK_DIR   = CRACEN_DIR / "work"     # батчі/маніфести/тимчасові файли
FINAL_PATH = CRACEN_DIR / "final.parquet"

CRACEN_DIR.mkdir(parents=True, exist_ok=True)
WORK_DIR.mkdir(parents=True, exist_ok=True)

BATCH_DIR = WORK_DIR / "CRACEN_batch"
BATCH_DIR.mkdir(parents=True, exist_ok=True)
ETF_DIR = WORK_DIR / "ETF"
ETF_DIR.mkdir(parents=True, exist_ok=True)


print("ORION_HOME:", ORION_HOME)
print("CRACEN_DIR :", CRACEN_DIR)
print("WORK_DIR  :", WORK_DIR)
print("FINAL_PATH:", FINAL_PATH)


ORION_HOME: C:\Users\sergi\OneDrive\Рабочий стол\ORION_MAIN\OriON
CRACEN_DIR : C:\Users\sergi\OneDrive\Рабочий стол\ORION_MAIN\OriON\CRACEN
WORK_DIR  : C:\Users\sergi\OneDrive\Рабочий стол\ORION_MAIN\OriON\CRACEN\work
FINAL_PATH: C:\Users\sergi\OneDrive\Рабочий стол\ORION_MAIN\OriON\CRACEN\final.parquet


In [5]:
from datetime import datetime, timedelta, date, time
from calendar import monthrange

# === ТЗ користувача: Europe/Kyiv ===
tz = None  # якщо нема zoneinfo, все одно працює локально

now_local = datetime.now(tz) if tz else datetime.now()
yesterday = (now_local - timedelta(days=1)).date()

# три місяці назад від учора (коректно для різної довжини місяців)
try:
    from dateutil.relativedelta import relativedelta
    start_date = (datetime.combine(yesterday, time(0, 0, 0)) - relativedelta(months=3)).date()
except Exception:
    # fallback без dateutil
    y, m, d = yesterday.year, yesterday.month - 3, yesterday.day
    while m <= 0:
        m += 12
        y -= 1
    d = min(d, monthrange(y, m)[1])
    start_date = date(y, m, d)

# формати з часом
start = f"{start_date:%Y-%m-%d} 00:00:00"
end   = f"{yesterday:%Y-%m-%d} 23:59:59"

# формати без часу
start_date_str = f"{start_date:%Y-%m-%d}"
end_date_str   = f"{yesterday:%Y-%m-%d}"

# вивід
print("🕓 Повні дати (з часом):")
print("start =", start)
print("end   =", end)
print("\n📅 Короткі дати (без часу):")
print("start_date_str =", start_date_str)
print("end_date_str   =", end_date_str)


🕓 Повні дати (з часом):
start = 2025-10-26 00:00:00
end   = 2026-01-26 23:59:59

📅 Короткі дати (без часу):
start_date_str = 2025-10-26
end_date_str   = 2026-01-26


In [6]:
import datetime

yesterday = (datetime.date.today() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")

params = {
    "date_as_of": yesterday,
    "format": "json_records"
}

df = DatumApi.data_request("/calculations/median_premarket_value_traded_ex_finr_3m", params)

print(df.head())
print("Рядків:", len(df))


Signature has expired


  ticker       value
0      A    53276.79
1     AA  1196849.64
2    AAA        0.00
3   AAAA        0.00
4   AAAC        0.00
Рядків: 12520


In [7]:
def filter_nonzero(df: pd.DataFrame) -> pd.DataFrame:
    """
    Повертає копію датафрейму лише з тими рядками,
    де колонка 'value' не дорівнює нулю.
    """
    if "value" not in df.columns:
        raise ValueError("У датафреймі немає колонки 'value'")
    return df[df["value"].fillna(0) != 0].copy()


# --- крок 1: фільтруємо df (де 'value' != 0)
df_filtered = filter_nonzero(df)

# --- крок 2: тягнемо дані по тікерах
ticker_params = {
    'fields': 'lvl3,country,exchange,market_cap',
    'active': True,
    'us_exchange': True,
    'listed': True
}
tickers_df = DatumApi.data_request('/tickers', ticker_params)
tickers_df = tickers_df.dropna()

# --- крок 3: додаємо потрібні колонки у df_filtered
top1500 = df_filtered.merge(
    tickers_df,
    how='left',
    on='ticker'
)

print(f"Після фільтрації залишилось: {len(top1500):,} рядків")
top1500.head()


Після фільтрації залишилось: 6,698 рядків


,ticker,value,lvl3,country,exchange,market_cap
0,A,53276.79,Medical Equipment & Devices,UNITED STATES,New York,38456.0
1,AA,1196849.64,Metals & Mining,UNITED STATES,New York,15162.0
2,AAAU,1291460.13,NaN,NaN,NaN,NaN
3,AACG,36.46,Software,UNITED STATES,NASDAQ CM,34.0
4,AACI,16875.54,NaN,NaN,NaN,NaN


In [8]:
top1500[top1500["ticker"] == "JD"]


,ticker,value,lvl3,country,exchange,market_cap
3215,JD,3277606.26,E-Commerce Discretionary,UNITED STATES,NASDAQ GS,40531.0


In [9]:
def get_unique_lvl2(df: pd.DataFrame) -> list:
    """
    Повертає список унікальних значень з колонки 'lvl3' у датафреймі.
    Пропуски (NaN) ігноруються.
    """
    if "lvl3" not in df.columns:
        raise ValueError("У датафреймі немає колонки 'lvl3'")
    
    unique_vals = sorted(df["lvl3"].dropna().unique().tolist())
    print(f"🔹 Знайдено {len(unique_vals)} унікальних значень у колонці 'lvl3'")
    return unique_vals


In [10]:
unique_lvl2 = get_unique_lvl2(top1500)
unique_lvl2


🔹 Знайдено 58 унікальних значень у колонці 'lvl3'


['Advertising & Marketing',
 'Aerospace & Defense',
 'Apparel & Textile Products',
 'Asset Management',
 'Automotive',
 'Banking',
 'Beverages',
 'Biotech & Pharma',
 'Cable & Satellite',
 'Chemicals',
 'Commercial Support Services',
 'Construction Materials',
 'Consumer Services',
 'Containers & Packaging',
 'Diversified Industrials',
 'E-Commerce Discretionary',
 'Electric Utilities',
 'Electrical Equipment',
 'Engineering & Construction',
 'Entertainment Content',
 'Food',
 'Forestry, Paper & Wood Products',
 'Gas & Water Utilities',
 'Health Care Facilities & Svcs',
 'Home & Office Products',
 'Home Construction',
 'Household Products',
 'IT Services',
 'Industrial Intermediate Prod',
 'Industrial Support Services',
 'Institutional Financial Svcs',
 'Insurance',
 'Internet Media & Services',
 'Leisure Facilities & Services',
 'Leisure Products',
 'Machinery',
 'Medical Equipment & Devices',
 'Metals & Mining',
 'Oil & Gas Services & Equip',
 'Oil & Gas Supply Chain',
 'Publishing &

In [11]:
sector_to_etf = {
    # --- Finance ---
    "Asset Management": ["XLF", "SPY", "BITO"],
    "Banking": ["KRE", "XLF", "SPY", "BITO"],
    "Institutional Financial Svcs": ["XLF", "SPY", "BITO"],
    "Insurance": ["XLF", "SPY", "BITO"],
    "Specialty Finance": ["XLF", "SPY", "BITO"],

    # --- Tech / Internet / Growth ---
    "Software": ["QQQ", "IWM", "SPY"],
    "IT Services": ["QQQ", "IWM", "SPY"],
    "Semiconductors": ["SOXL", "QQQ", "IWM", "SPY"],
    "Technology Hardware": ["QQQ", "SOXL", "IWM", "SPY"],
    "Internet Media & Services": ["QQQ", "KWEB", "IWM", "SPY"],
    "E-Commerce Discretionary": ["QQQ", "KWEB", "IWM", "SPY"],
    "Cable & Satellite": ["QQQ", "KWEB", "IWM", "SPY"],
    "Telecommunications": ["QQQ", "KWEB", "IWM", "SPY"],
    "Publishing & Broadcasting": ["QQQ", "KWEB", "IWM", "SPY"],
    "Entertainment Content": ["QQQ", "KWEB", "IWM", "SPY"],
    "Advertising & Marketing": ["QQQ", "KWEB", "IWM", "SPY"],

    # --- Industrials ---
    "Aerospace & Defense": ["IWM", "SPY", "QQQ"],
    "Diversified Industrials": ["SPY", "IWM"],
    "Industrial Intermediate Prod": ["IWM", "SPY"],
    "Industrial Support Services": ["IWM", "SPY"],
    "Engineering & Construction": ["IWM", "SPY"],
    "Machinery": ["IWM", "SPY"],
    "Transportation & Logistics": ["IWM", "SPY"],
    "Transportation Equipment": ["IWM", "SPY"],
    "Commercial Support Services": ["IWM", "SPY"],
    "Containers & Packaging": ["IWM", "SPY"],
    "Electrical Equipment": ["QQQ", "IWM", "SPY"],

    # --- Energy / Materials / Commodities ---
    "Oil & Gas Services & Equip": ["XLE", "IWM", "SPY"],
    "Oil & Gas Supply Chain": ["XLE", "IWM", "SPY"],
    "Elec & Gas Marketing & Trading": ["XLE", "IWM", "SPY"],
    "Electric Utilities": ["XLE", "IWM", "SPY"],
    "Gas & Water Utilities": ["XLE", "IWM", "SPY"],
    "Renewable Energy": ["XLE", "IWM", "SPY"],
    "Chemicals": ["XLE", "IWM", "SPY"],
    "Metals & Mining": ["GDX", "IWM", "SPY"],
    "Steel": ["GDX", "IWM", "SPY"],
    "Construction Materials": ["IWM", "SPY"],
    "Forestry, Paper & Wood Products": ["IWM", "SPY"],

    # --- Health Care ---
    "Biotech & Pharma": ["QQQ", "IWM", "SPY"],
    "Medical Equipment & Devices": ["QQQ", "IWM", "SPY"],
    "Health Care Facilities & Svcs": ["QQQ", "IWM", "SPY"],

    # --- Consumer Staples / Discretionary ---
    "Food": ["SPY", "IWM"],
    "Beverages": ["SPY", "IWM"],
    "Household Products": ["SPY", "IWM"],
    "Tobacco & Cannabis": ["SPY", "IWM"],
    "Retail - Consumer Staples": ["SPY", "IWM"],
    "Wholesale - Consumer Staples": ["SPY", "IWM"],

    "Retail - Discretionary": ["SPY", "IWM", "QQQ"],
    "Wholesale - Discretionary": ["SPY", "IWM", "QQQ"],
    "Consumer Services": ["SPY", "IWM"],
    "Leisure Facilities & Services": ["SPY", "IWM"],
    "Leisure Products": ["SPY", "IWM"],
    "Apparel & Textile Products": ["SPY", "IWM", "QQQ"],
    "Automotive": ["SPY", "IWM", "QQQ"],
    "Home & Office Products": ["SPY", "IWM"],
    "Home Construction": ["IWM", "SPY"],

    # --- Real Estate ---
    "REIT": ["IWM", "SPY"],
    "Real Estate Owners & Developers": ["IWM", "SPY"],
    "Real Estate Services": ["IWM", "SPY"],
}


In [12]:
# ------------------------------------------------

In [13]:
import time
import math
import pandas as pd

def corr_beta_best_match_batched(
    x_tickers,
    y_tickers,
    period="6 months",
    as_pivot=False,
    fmt="json_records",
    batch_size=150,
    pause=0.15,
    max_retries=3,
    output_path=None,          # 🔹 тепер дефолт = None
):
    """
    Викликає /calculations/corr_beta/v2 батчами, об'єднує результати в один DataFrame,
    для кожного X-тікера знаходить Y з максимальною кореляцією.

    Повертає (df_all, df_pairs):
      - df_all   : сирий об'єднаний датафрейм з API
      - df_pairs : мапінг X -> найкращий Y
    """
    # --- валідації ---
    if not x_tickers:
        empty_pairs = pd.DataFrame(
            columns=["x_ticker", "best_y_ticker", "best_corr", "beta_with_best"]
        )
        if output_path:
            empty_pairs.to_csv(output_path, index=False)
        return pd.DataFrame(), empty_pairs

    if not y_tickers:
        raise ValueError("y_tickers не може бути порожнім")

    # --- батчинг (індексний підхід, щоб не губити/дублювати X) ---
    y_str = ",".join(map(str, y_tickers))
    results = []
    i = 0
    n = len(x_tickers)
    bs = max(1, int(batch_size))

    while i < n:
        xs = list(map(str, x_tickers[i : i + bs]))
        x_str = ",".join(xs)

        params = {
            "x_tickers": x_str,
            "y_tickers": y_str,
            "period": period,
            "as_pivot": as_pivot,
            "format": fmt,
        }

        attempt = 0
        while True:
            try:
                df_chunk = DatumApi.data_request("/calculations/corr_beta/v2", params)

                # підстраховка типів
                if df_chunk is None:
                    fetched_ok = False
                elif isinstance(df_chunk, pd.DataFrame):
                    fetched_ok = not df_chunk.empty
                else:
                    df_chunk = pd.DataFrame(df_chunk)
                    fetched_ok = not df_chunk.empty

                if fetched_ok:
                    results.append(df_chunk)

                i += len(xs)
                print(f"[{i}/{n}] fetched batch of {len(xs)} X тикерів")
                time.sleep(pause)
                break

            except Exception as e:
                msg = str(e).lower()
                attempt += 1

                if "414" in msg or "uri too large" in msg or "request-uri too large" in msg:
                    new_bs = max(1, bs // 2)
                    if new_bs == bs:
                        if attempt >= max_retries:
                            print(
                                f"⚠️  Пропускаю {len(xs)} X-тикерів (i={i}) "
                                f"після {max_retries} 414-помилок. Помилка: {e}"
                            )
                            i += len(xs)
                            break
                    else:
                        print(
                            f"⚠️  414 на i={i}. Зменшую batch_size {bs} → {new_bs} "
                            f"і формую партію заново."
                        )
                        bs = new_bs
                        break
                else:
                    if attempt >= max_retries:
                        print(
                            f"⚠️  Пропускаю {len(xs)} X-тикерів (i={i}) "
                            f"після {max_retries} спроб. Помилка: {e}"
                        )
                        i += len(xs)
                        break
                    time.sleep(0.5 * attempt)

    df_all = pd.concat(results, ignore_index=True) if results else pd.DataFrame()

    # --- нормалізація назв колонок ---
    def _col(options, cols_lower):
        for c in options:
            if c.lower() in cols_lower:
                return next(cc for cc in df_all.columns if cc.lower() == c.lower())
        return None

    if df_all.empty:
        empty_pairs = pd.DataFrame(
            columns=["x_ticker", "best_y_ticker", "best_corr", "beta_with_best"]
        )
        if output_path:
            empty_pairs.to_csv(output_path, index=False)
        return df_all, empty_pairs

    cols_lower = {c.lower() for c in df_all.columns}

    x_col    = _col(["x_ticker", "x", "ticker_x", "subject", "symbol_x"], cols_lower)
    y_col    = _col(["y_ticker", "y", "ticker_y", "benchmark", "symbol_y"], cols_lower)
    corr_col = _col(["corr", "correlation", "pearson_corr"], cols_lower)
    beta_col = _col(["beta", "beta_coeff"], cols_lower)

    missing = [
        name
        for name, val in [("x_ticker", x_col), ("y_ticker", y_col), ("corr", corr_col)]
        if val is None
    ]
    if missing:
        raise ValueError(
            f"Від API бракує очікуваних колонок: {', '.join(missing)}. "
            f"Є колонки: {list(df_all.columns)}"
        )

    # --- вибір найкращого Y для кожного X за corr ---
    df_sorted = df_all.sort_values(by=corr_col, ascending=False)
    idx_best = df_sorted.groupby(x_col, as_index=False)[corr_col].idxmax()[corr_col]
    df_best = df_sorted.loc[
        idx_best, [x_col, y_col, corr_col] + ([beta_col] if beta_col else [])
    ].copy()

    rename_map = {
        x_col: "x_ticker",
        y_col: "best_y_ticker",
        corr_col: "best_corr",
    }
    if beta_col:
        rename_map[beta_col] = "beta_with_best"
    df_pairs = df_best.rename(columns=rename_map)

    if "beta_with_best" not in df_pairs.columns:
        df_pairs["beta_with_best"] = pd.NA

    df_pairs = df_pairs[["x_ticker", "best_y_ticker", "best_corr", "beta_with_best"]]

    if output_path:
        df_pairs.to_csv(output_path, index=False)
        print(f"✅ Збережено {len(df_pairs)} пар у файл: {output_path}")

    return df_all, df_pairs


In [14]:
def corr_beta_best_etf_by_lvl3(
    tickers_df,
    sector_to_etf,
    sector_col="lvl3",
    ticker_col="ticker",
    period="6 months",
    as_pivot=False,
    fmt="json_records",
    batch_size=150,
    pause=0.15,
    max_retries=3,
    output_path="CRACEN/work/x_to_best_etf_by_lvl3.csv",
    bench_path="ticker_bench.csv",
    fallback_etfs=None,
):
    """
    Створює два файли:
      1) output_path (за замовчуванням x_to_best_etf_by_lvl3.csv):
         x_ticker, best_y_ticker, best_corr, beta_with_best, lvl3

      2) bench_path (за замовчуванням ticker_bench.csv):
         ticker, benchmark

    Логіка:
      - Для кожного lvl3-сектора збираємо X-список тікерів.
      - Y-список ETF-ів беремо з sector_to_etf[lvl3].
      - Якщо сектору немає в мапі або список порожній → використовуємо fallback_etfs
        (за замовчуванням ["SPY", "QQQ", "IWM"]).
      - Для кожного тикера залишаємо ETF з максимальною кореляцією.
      - Тікери без lvl3 (NaN) обробляються як сектор 'UNKNOWN' з fallback ETF-ами.
    """

    import pandas as pd  # на випадок, якщо не імпортнуто вище

    if fallback_etfs is None:
        fallback_etfs = ["SPY", "QQQ", "IWM"]

    if sector_col not in tickers_df.columns:
        raise ValueError(f"У tickers_df немає колонки '{sector_col}'")
    if ticker_col not in tickers_df.columns:
        raise ValueError(f"У tickers_df немає колонки '{ticker_col}'")

    # Беремо лише тикер і сектор, але НЕ дропаємо NaN — замінимо їх на 'UNKNOWN'
    df_clean = (
        tickers_df[[ticker_col, sector_col]]
        .drop_duplicates()
        .copy()
    )

    # 🔥 Тікери без сектора → 'UNKNOWN'
    df_clean[sector_col] = df_clean[sector_col].fillna("UNKNOWN").astype(str)
    df_clean[ticker_col] = df_clean[ticker_col].astype(str)

    all_sectors = sorted(df_clean[sector_col].unique().tolist())

    all_chunks = []
    all_pairs = []
    sectors_with_fallback = []

    for sector in all_sectors:
        etfs = sector_to_etf.get(sector)

        if not etfs:
            etfs = list(fallback_etfs)
            sectors_with_fallback.append(sector)
            print(
                f"⚠️  Для сектора '{sector}' немає ETF-ів у sector_to_etf — "
                f"використовую fallback: {etfs}"
            )
        else:
            print(f"✅ lvl3: {sector} → ETF-и з мапи: {etfs}")

        x_tickers = (
            df_clean.loc[df_clean[sector_col] == sector, ticker_col]
            .astype(str)
            .unique()
            .tolist()
        )
        if not x_tickers:
            continue

        print(f"\n=== lvl3: {sector} | X-тикерів: {len(x_tickers)} | ETF-и: {etfs} ===")

        df_all_sec, df_pairs_sec = corr_beta_best_match_batched(
            x_tickers=x_tickers,
            y_tickers=etfs,
            period=period,
            as_pivot=as_pivot,
            fmt=fmt,
            batch_size=batch_size,
            pause=pause,
            max_retries=max_retries,
            output_path=None,   # тут не пишемо окремі файли
        )

        if not df_all_sec.empty:
            df_all_sec = df_all_sec.copy()
            df_all_sec[sector_col] = sector
            all_chunks.append(df_all_sec)

        if not df_pairs_sec.empty:
            df_pairs_sec = df_pairs_sec.copy()
            df_pairs_sec[sector_col] = sector
            all_pairs.append(df_pairs_sec)

    df_all = pd.concat(all_chunks, ignore_index=True) if all_chunks else pd.DataFrame()

    if all_pairs:
        df_pairs = pd.concat(all_pairs, ignore_index=True)

        # на випадок дублів по x_ticker — лишаємо найбільшу кореляцію
        df_pairs_sorted = df_pairs.sort_values(by="best_corr", ascending=False)
        idx_best = (
            df_pairs_sorted.groupby("x_ticker", as_index=False)["best_corr"]
            .idxmax()["best_corr"]
        )
        df_pairs = df_pairs_sorted.loc[idx_best].reset_index(drop=True)

        # ⚙️ детальний файл
        df_pairs.to_csv(output_path, index=False)
        print(f"\n✅ Збережено {len(df_pairs)} X→ETF пар у файл: {output_path}")

        # ⚙️ спрощений файл вигляду ticker,benchmark
        df_bench = df_pairs.rename(
            columns={"x_ticker": "ticker", "best_y_ticker": "benchmark"}
        )[["ticker", "benchmark"]]

        df_bench.to_csv(bench_path, index=False)
        print(f"✅ Збережено {len(df_bench)} записів у файл: {bench_path}")

    else:
        df_pairs = pd.DataFrame(
            columns=["x_ticker", "best_y_ticker", "best_corr", "beta_with_best", sector_col]
        )
        df_pairs.to_csv(output_path, index=False)
        pd.DataFrame(columns=["ticker", "benchmark"]).to_csv(bench_path, index=False)
        print("\n⚠️ Не вдалося знайти жодної пари X→ETF. Створено порожні файли.")

    if sectors_with_fallback:
        print("\n⚠️ Для цих lvl3 використано fallback ETF-и (SPY, QQQ, IWM):")
        for s in sectors_with_fallback:
            print("   -", s)

    return df_all, df_pairs


In [15]:
# нове: працюємо напряму з таблицею top1500 (ticker + lvl3)
df_all, df_pairs = corr_beta_best_etf_by_lvl3(
    tickers_df=top1500,          # DataFrame з колонками ticker, lvl3, ...
    sector_to_etf=sector_to_etf, # мапа lvl3 -> список ETF-ів
    sector_col="lvl3",           # Yазва колонки сектора в top1500
    ticker_col="ticker",         # назва колонки з тикером
    period="3 months",
    as_pivot=False,
    fmt="json_records",
    batch_size=150,
    pause=0.1,
    max_retries=3,
    output_path="CRACEN/work/x_to_best_etf_by_lvl3.csv",  # детальна інфа
    bench_path=str(WORK_DIR / "ticker_bench_.csv")
            # простий файл ticker,benchmark
)

print(df_all.head())
print("Рядків у сирих даних:", len(df_all))
print(df_pairs.head())


✅ lvl3: Advertising & Marketing → ETF-и з мапи: ['QQQ', 'KWEB', 'IWM', 'SPY']

=== lvl3: Advertising & Marketing | X-тикерів: 37 | ETF-и: ['QQQ', 'KWEB', 'IWM', 'SPY'] ===


[37/37] fetched batch of 37 X тикерів
✅ lvl3: Aerospace & Defense → ETF-и з мапи: ['IWM', 'SPY', 'QQQ']

=== lvl3: Aerospace & Defense | X-тикерів: 69 | ETF-и: ['IWM', 'SPY', 'QQQ'] ===


[69/69] fetched batch of 69 X тикерів
✅ lvl3: Apparel & Textile Products → ETF-и з мапи: ['SPY', 'IWM', 'QQQ']

=== lvl3: Apparel & Textile Products | X-тикерів: 36 | ETF-и: ['SPY', 'IWM', 'QQQ'] ===


[36/36] fetched batch of 36 X тикерів
✅ lvl3: Asset Management → ETF-и з мапи: ['XLF', 'SPY', 'BITO']

=== lvl3: Asset Management | X-тикерів: 194 | ETF-и: ['XLF', 'SPY', 'BITO'] ===


[150/194] fetched batch of 150 X тикерів


...

[1950/2031] fetched batch of 150 X тикерів


[2031/2031] fetched batch of 81 X тикерів
✅ lvl3: Wholesale - Consumer Staples → ETF-и з мапи: ['SPY', 'IWM']

=== lvl3: Wholesale - Consumer Staples | X-тикерів: 12 | ETF-и: ['SPY', 'IWM'] ===


[12/12] fetched batch of 12 X тикерів
✅ lvl3: Wholesale - Discretionary → ETF-и з мапи: ['SPY', 'IWM', 'QQQ']

=== lvl3: Wholesale - Discretionary | X-тикерів: 14 | ETF-и: ['SPY', 'IWM', 'QQQ'] ===


[14/14] fetched batch of 14 X тикерів



✅ Збережено 6510 X→ETF пар у файл: CRACEN/work/x_to_best_etf_by_lvl3.csv
✅ Збережено 6510 записів у файл: C:\Users\sergi\OneDrive\Рабочий стол\ORION_MAIN\OriON\CRACEN\work\ticker_bench_.csv

⚠️ Для цих lvl3 використано fallback ETF-и (SPY, QQQ, IWM):
   - UNKNOWN
  x_ticker y_ticker  corr  beta                     lvl3
0      ADV      SPY  0.60  0.62  Advertising & Marketing
1      ADV      IWM  0.56  0.50  Advertising & Marketing
2      ADV      QQQ  0.51  0.00  Advertising & Marketing
3      ADV     KWEB  0.13  0.00  Advertising & Marketing
4      APP      SPY  0.95  2.84  Advertising & Marketing
Рядків у сирих даних: 19154
  x_ticker best_y_ticker  best_corr  beta_with_best  \
0        A           IWM       0.75            0.55   
1       AA           SPY       0.70            1.93   
2     AAAU           IWM       0.34            0.23   
3     AACG           IWM       0.18            0.00   
4      AAL           SPY       0.86            1.50   

                          lvl3  
0

In [16]:
# ================== FAST INTRADAY SUITE w/ BLUE OCEAN (updated + rolling prune) ==================
import os, gc, time, glob, re, csv, uuid, random
import pandas as pd
from datetime import timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Optional, Tuple

# ---------- utils ----------
def _p(*args, **kwargs):
    kwargs.setdefault("flush", True)
    print(*args, **kwargs)

_PSUTIL = False
try:
    import psutil
    _PSUTIL = True
except Exception:
    psutil = None

def _proc_rss_mb() -> float:
    if _PSUTIL:
        try:
            return psutil.Process(os.getpid()).memory_info().rss / (1024**2)
        except Exception:
            pass
    try:
        import resource
        ru = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
        if ru > 10**9:
            return ru / (1024**2)
        return ru / 1024.0
    except Exception:
        pass
    try:
        import tracemalloc
        if not tracemalloc.is_tracing():
            tracemalloc.start()
        cur, _ = tracemalloc.get_traced_memory()
        return cur / (1024**2)
    except Exception:
        return float('nan')

def _mem_str(prefix: str = "") -> str:
    mb = _proc_rss_mb()
    if mb != mb:
        return f"{prefix}🧠 RAM: n/a"
    return f"{prefix}🧠 RAM: {mb:,.1f} MB"

def _ensure_dir(p: str):
    os.makedirs(p, exist_ok=True)

def _fmt_eta(sec: float) -> str:
    sec = max(0, int(sec))
    d, r = divmod(sec, 86400)
    h, r = divmod(r, 3600)
    m, s = divmod(r, 60)
    if d > 0: return f"{d}d {h:02d}:{m:02d}:{s:02d}"
    if h > 0: return f"{h:02d}:{m:02d}:{s:02d}"
    return f"{m:02d}:{s:02d}"

def _next_batch_index(out_dir: str, out_prefix: str) -> int:
    pat_csv = os.path.join(out_dir, f"{out_prefix}_*.csv")
    pat_gz  = os.path.join(out_dir, f"{out_prefix}_*.csv.gz")
    files = glob.glob(pat_csv) + glob.glob(pat_gz)
    mx = 0
    rx = re.compile(rf'^{re.escape(out_prefix)}_(\d+)\.csv(?:\.gz)?$')
    for f in files:
        m = rx.match(os.path.basename(f))
        if m:
            mx = max(mx, int(m.group(1)))
    return mx + 1 if mx > 0 else 1

# ---------- manifest helpers (old ticker-level) ----------
def _manifest_path(out_dir: str) -> str:
    return os.path.join(out_dir, "done_tickers.txt")

def _load_done_tickers_manifest(out_dir: str) -> Optional[set]:
    path = _manifest_path(out_dir)
    if not os.path.exists(path):
        return None
    done = set()
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            t = line.strip()
            if t:
                done.add(t)
    return done

def _append_done_tickers_manifest(out_dir: str, tickers: List[str]):
    if not tickers:
        return
    path = _manifest_path(out_dir)
    existing = set()
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                s = line.strip()
                if s:
                    existing.add(s)
    new_ones = [t for t in set(tickers) if t not in existing]
    if not new_ones:
        return
    with open(path, "a", encoding="utf-8") as f:
        for t in new_ones:
            f.write(t + "\n")

# ---------- day-manifest helpers (NEW) ----------
def _manifest_days_path(out_dir: str) -> str:
    return os.path.join(out_dir, "done_days.csv")

def _load_done_days_manifest(out_dir: str) -> dict:
    """
    Return dict: ticker -> set({'YYYY-MM-DD', ...})
    """
    path = _manifest_days_path(out_dir)
    if not os.path.exists(path):
        return {}
    done = {}
    with open(path, "r", encoding="utf-8", newline="") as f:
        rdr = csv.reader(f)
        header = next(rdr, None)
        col_idx = {"ticker": 0, "date": 1}
        if header and "ticker" in header and "date" in header:
            col_idx["ticker"] = header.index("ticker")
            col_idx["date"]   = header.index("date")
        else:
            f.seek(0); rdr = csv.reader(f)
        for row in rdr:
            if not row:
                continue
            tk = row[col_idx["ticker"]].strip()
            dt = row[col_idx["date"]].strip()
            if tk and dt:
                done.setdefault(tk, set()).add(dt)
    return done

def _append_done_days_manifest(out_dir: str, df: pd.DataFrame):
    """
    From df uses 'ticker' and 'dt' (YYYY-MM-DD HH:MM:SS) to append (ticker,date) to done_days.csv
    """
    if df is None or df.empty or "dt" not in df.columns or "ticker" not in df.columns:
        return
    path = _manifest_days_path(out_dir)
    _ensure_dir(out_dir)

    tmp = df[["ticker","dt"]].copy()
    tmp["date"] = tmp["dt"].astype(str).str.slice(0,10)
    tmp = tmp.drop_duplicates(subset=["ticker","date"])

    need_header = not os.path.exists(path)
    with open(path, "a", encoding="utf-8", newline="") as f:
        wr = csv.writer(f)
        if need_header:
            wr.writerow(["ticker","date"])
        for _, r in tmp.iterrows():
            wr.writerow([str(r["ticker"]).strip(), str(r["date"]).strip()])

def prune_done_days_manifest(out_dir: str, start_date_str: str, end_date_str: str) -> dict:
    """
    Rolling-window prune for done_days.csv:
    - keep only rows where start_date_str <= date <= end_date_str
    - returns refreshed dict: ticker -> set(dates)
    Safe to call even if file doesn't exist.
    """
    path = _manifest_days_path(out_dir)
    if not os.path.exists(path):
        return {}

    # normalize inputs
    start_date_str = str(start_date_str)[:10]
    end_date_str   = str(end_date_str)[:10]
    if start_date_str > end_date_str:
        # swap just in case
        start_date_str, end_date_str = end_date_str, start_date_str

    rows = []
    with open(path, "r", encoding="utf-8", newline="") as f:
        rdr = csv.reader(f)
        header = next(rdr, None)
        col_idx = {"ticker": 0, "date": 1}
        has_header = bool(header and "ticker" in header and "date" in header)
        if has_header:
            col_idx["ticker"] = header.index("ticker")
            col_idx["date"]   = header.index("date")
        else:
            # no header; treat first row as data
            if header:
                rdr = [header] + list(rdr)  # include first row back as data
            else:
                rdr = []
        for row in rdr:
            if not row:
                continue
            try:
                tk = str(row[col_idx["ticker"]]).strip().upper()
                dt = str(row[col_idx["date"]]).strip()[:10]
            except Exception:
                continue
            if not tk or not dt:
                continue
            if start_date_str <= dt <= end_date_str:
                rows.append((tk, dt))

    # rewrite file (dedup)
    _ensure_dir(out_dir)
    tmp_path = path + ".tmp"
    seen = set()
    with open(tmp_path, "w", encoding="utf-8", newline="") as f:
        wr = csv.writer(f)
        wr.writerow(["ticker","date"])
        for tk, dt in rows:
            key = (tk, dt)
            if key in seen:
                continue
            seen.add(key)
            wr.writerow([tk, dt])
    os.replace(tmp_path, path)

    # build dict cache
    done = {}
    for tk, dt in seen:
        done.setdefault(tk, set()).add(dt)
    return done

def _dates_between(start: str, end: str) -> List[pd.Timestamp]:
    s = pd.to_datetime(start)
    e = pd.to_datetime(end)
    if pd.isna(s) or pd.isna(e) or s > e:
        return []
    days = pd.date_range(start=s.normalize(), end=e.normalize(), freq="D")
    return list(days)

def _group_consecutive_dates(dates: List[pd.Timestamp], max_span_days: int) -> List[Tuple[str, str]]:
    """
    Build [start_dt, end_dt] ranges from a list of dates, merging consecutive days,
    capped by max_span_days per range. Datetimes are ISO to seconds.
    """
    if not dates:
        return []
    dates = sorted(pd.to_datetime(dates))
    out = []
    run_start = dates[0]
    run_end = dates[0]
    for d in dates[1:]:
        consecutive = (d - run_end).days == 1
        span_len = (d - run_start).days + 1
        if consecutive and span_len <= max_span_days:
            run_end = d
        else:
            a = run_start.strftime("%Y-%m-%d 00:00:00")
            b = (run_end + pd.Timedelta(hours=23, minutes=59, seconds=59)).strftime("%Y-%m-%d %H:%M:%S")
            out.append((a, b))
            run_start = d
            run_end = d
    a = run_start.strftime("%Y-%m-%d 00:00:00")
    b = (run_end + pd.Timedelta(hours=23, minutes=59, seconds=59)).strftime("%Y-%m-%d %H:%M:%S")
    out.append((a, b))
    return out

# ================== SMALL (low-level) FUNCTIONS ==================
def _build_ranges(start: str, end: str, chunk_days: int) -> List[Tuple[str, str]]:
    """Розбиває [start,end] на неперекривні відрізки по chunk_days днів."""
    chunk_days = max(1, int(chunk_days))
    s = pd.to_datetime(start)
    e = pd.to_datetime(end)
    if pd.isna(s) or pd.isna(e) or s > e:
        return []
    out: List[Tuple[str, str]] = []
    cur = s
    step = timedelta(days=chunk_days) - timedelta(seconds=1)
    while cur <= e:
        r_end = min(cur + step, e)
        out.append((cur.strftime("%Y-%m-%d %H:%M:%S"), r_end.strftime("%Y-%m-%d %H:%M:%S")))
        cur = r_end + timedelta(seconds=1)
    return out

def _one_call_v3_dt_fast(
    ticker: str,
    start_dt: str,
    end_dt: str,
    interval: int,
    *,
    chart_type: str = "ohlcv",
    include_boats: bool = False,
    max_retries: int = 2,
    retry_backoff: float = 0.5,     # сек
    normalize_ohlcv: bool = False,  # дефолт: ВИМКНЕНО (швидкість)
) -> pd.DataFrame:
    """Основний ендпоінт: /intraday/v3."""
    params = {
        "ticker": ticker.upper(),
        "interval": int(interval),
        "chart_type": chart_type,
        "start_datetime": start_dt,
        "end_datetime": end_dt,
        "include_boats": bool(include_boats),
        "format": "json_records",
    }
    attempts = max(0, int(max_retries)) + 1
    for attempt in range(attempts):
        try:
            df = DatumApi.data_request("/intraday/v3", params)
            if isinstance(df, pd.DataFrame) and not df.empty:
                if "ticker" not in df.columns:
                    df.insert(0, "ticker", ticker.upper())
                if normalize_ohlcv:
                    for col in ("o","h","l","c","v"):
                        if col in df.columns:
                            df[col] = pd.to_numeric(df[col], errors="ignore")
                return df
            return pd.DataFrame()
        except Exception:
            if attempt == attempts - 1:
                return pd.DataFrame()
            sleep_s = retry_backoff * (2 ** attempt)
            sleep_s *= (0.9 + 0.2 * random.random())
            time.sleep(sleep_s)

def _one_call_blue_ocean_v3_dt_fast(
    ticker: str,
    start_dt: str,
    end_dt: str,
    interval: int,
    *,
    chart_type: str = "ohlcv",
    max_retries: int = 2,
    retry_backoff: float = 0.5,
    normalize_ohlcv: bool = False,
) -> pd.DataFrame:
    """Blue Ocean ендпоінт: /intraday/v3/blue_ocean (20:00–04:00)."""
    params = {
        "ticker": ticker.upper(),
        "interval": int(interval),
        "chart_type": chart_type,
        "start_datetime": start_dt,
        "end_datetime": end_dt,
        "format": "json_records",
    }
    attempts = max(0, int(max_retries)) + 1
    for attempt in range(attempts):
        try:
            df = DatumApi.data_request("/intraday/v3/blue_ocean", params)
            if isinstance(df, pd.DataFrame) and not df.empty:
                if "ticker" not in df.columns:
                    df.insert(0, "ticker", ticker.upper())
                if normalize_ohlcv:
                    for col in ("o","h","l","c","v"):
                        if col in df.columns:
                            df[col] = pd.to_numeric(df[col], errors="ignore")
                return df
            return pd.DataFrame()
        except Exception:
            if attempt == attempts - 1:
                return pd.DataFrame()
            sleep_s = retry_backoff * (2 ** attempt)
            sleep_s *= (0.9 + 0.2 * random.random())
            time.sleep(sleep_s)

def fetch_intraday_v3_datetime_fast(
    ticker: str,
    start: str,
    end: str,
    interval: int = 1,
    *,
    chunk_days: int = 14,
    parallel: bool = True,
    max_workers: int = 12,
    chart_type: str = "ohlcv",
    include_boats: bool = False,
    sort_by_dt: bool = False,   # дефолт: викл (швидкість)
    dedup_dt: bool = False,     # дефолт: викл (швидкість)
    normalize_ohlcv: bool = False,  # дефолт: викл (швидкість)
    include_blue_ocean: bool = False,   # ⬅️ НОВЕ
) -> pd.DataFrame:
    """
    Тягне інтрадійні дані основного вікна і, опційно, Blue Ocean (20:00–04:00),
    зшиває в один DF. За замовчуванням без зайвих перетворень.
    """
    ranges = _build_ranges(start, end, chunk_days)
    if not ranges:
        return pd.DataFrame()

    worker_count = max(1, min(int(max_workers), len(ranges) * (2 if include_blue_ocean else 1))) if parallel else 1
    parts: List[pd.DataFrame] = []

    if parallel and (len(ranges) > 1 or include_blue_ocean):
        with ThreadPoolExecutor(max_workers=worker_count) as ex:
            futs = []
            for (a, b) in ranges:
                futs.append(ex.submit(
                    _one_call_v3_dt_fast, ticker, a, b, interval,
                    chart_type=chart_type, include_boats=include_boats,
                    normalize_ohlcv=normalize_ohlcv
                ))
                if include_blue_ocean:
                    futs.append(ex.submit(
                        _one_call_blue_ocean_v3_dt_fast, ticker, a, b, interval,
                        chart_type=chart_type, normalize_ohlcv=normalize_ohlcv
                    ))
            for f in as_completed(futs):
                df = f.result()
                if df is not None and not df.empty:
                    parts.append(df)
    else:
        for (a, b) in ranges:
            df_main = _one_call_v3_dt_fast(
                ticker, a, b, interval,
                chart_type=chart_type, include_boats=include_boats,
                normalize_ohlcv=normalize_ohlcv
            )
            if df_main is not None and not df_main.empty:
                parts.append(df_main)
            if include_blue_ocean:
                df_bo = _one_call_blue_ocean_v3_dt_fast(
                    ticker, a, b, interval,
                    chart_type=chart_type, normalize_ohlcv=normalize_ohlcv
                )
                if df_bo is not None and not df_bo.empty:
                    parts.append(df_bo)

    if not parts:
        return pd.DataFrame()

    out = pd.concat(parts, ignore_index=True)

    # Якщо підмішували Blue Ocean — відсортуємо по 'dt'
    if "dt" in out.columns and (include_blue_ocean or sort_by_dt or dedup_dt):
        if dedup_dt:
            out = out.drop_duplicates(subset=["ticker", "dt"], keep="last")
        out = out.sort_values("dt", kind="mergesort").reset_index(drop=True)

    return out

# ================== BIG (all tickers, batch writer) [UPDATED + rolling prune] ==================
def fetch_intraday_v3_for_all_tickers(
    tickers_df: pd.DataFrame,
    start: str,
    end: str,
    *,
    interval: int = 1,
    chunk_days: int = 30,
    parallel_chunks: bool = True,
    max_workers_chunks: int = 12,
    parallel_tickers: bool = True,
    max_workers_tickers: int = 6,
    save_every: int = 50,
    out_dir: str = "intraday_batches",
    out_prefix: str = "batch",
    gzip: bool = True,
    keep_columns: Optional[List[str]] = None,
    verbose: bool = True,
    progress_every: int = 10,
    mem_log_every: int = 50,
    use_manifest: bool = False,            # old ticker-level manifest (off by default)
    use_day_manifest: bool = True,         # NEW: day-level manifest (on by default)
    flush_when_rows: Optional[int] = None,
    flush_every_seconds: Optional[int] = None,
    include_blue_ocean: bool = False,      # pass-through
) -> pd.DataFrame:
    assert "ticker" in tickers_df.columns, "tickers_df має містити колонку 'ticker'"

    # ⬆️ НОРМАЛІЗАЦІЯ: тікери у верхній регістр одразу на вході
    tickers: List[str] = [str(t).strip().upper() for t in tickers_df["ticker"].dropna().unique()]
    if not tickers:
        return pd.DataFrame(columns=["ticker","error"])

    _ensure_dir(out_dir)

    # --- rolling window bounds as YYYY-MM-DD for manifest prune ---
    start_date_str = str(start)[:10]
    end_date_str   = str(end)[:10]

    has_batches = bool(glob.glob(os.path.join(out_dir, f"{out_prefix}_*.csv*")))
    done_tickers = set()
    if use_manifest:
        m = _load_done_tickers_manifest(out_dir)
        if m and has_batches:
            done_tickers = m
        elif m and not has_batches and verbose:
            _p("⚠️ Знайшов done_tickers.txt, але немає batch-файлів → ігнорую маніфест.")
        # apply old manifest only if explicitly enabled
        tickers = [t for t in tickers if t not in done_tickers]

    # NEW: day manifest load (+ rolling prune)
    if use_day_manifest:
        # prune done_days.csv to the current rolling window and get fresh dict
        done_days_by_ticker = prune_done_days_manifest(out_dir, start_date_str, end_date_str)
    else:
        done_days_by_ticker = {}

    # ⬆️ НОРМАЛІЗАЦІЯ: ключі маніфесту у верхній регістр
    if use_day_manifest and done_days_by_ticker:
        done_days_by_ticker = {str(k).upper(): set(v) for k, v in done_days_by_ticker.items()}

    # --- попереднє відсікання повністю готових тікерів ---
    all_days_list = _dates_between(start, end)
    all_days_str = [d.strftime("%Y-%m-%d") for d in all_days_list]

    def _ticker_fully_done(tk: str) -> bool:
        if not use_day_manifest:
            return False
        s = done_days_by_ticker.get(str(tk).upper(), set())
        return bool(s) and all(dd in s for dd in all_days_str)

    fully_done = [t for t in tickers if _ticker_fully_done(t)]
    tickers = [t for t in tickers if t not in fully_done]

    total = len(tickers)
    batch_idx = _next_batch_index(out_dir, out_prefix)
    run_id = str(uuid.uuid4())[:8]

    errors, buffer = [], []
    written_total = 0
    start_time = time.time()
    last_flush_at = start_time
    rows_in_buffer = 0

    if verbose:
        psutil_note = "psutil" if _PSUTIL else "fallback"

        # логіка визначення діапазонів дат для логів
        if use_day_manifest and done_days_by_ticker:
            # усі дні в маніфесті
            all_done_dates = sorted({d for s in done_days_by_ticker.values() for d in s})
            skipped_days_str = (
                f"{all_done_dates[0]} → {all_done_dates[-1]} ({len(all_done_dates)} днів)"
                if all_done_dates else "–"
            )
            # усі дні, які ще не оброблені (порівняно з повним діапазоном)
            all_days_full = [d.strftime("%Y-%m-%d") for d in _dates_between(start, end)]
            remaining_days = sorted(set(all_days_full) - set(all_done_dates))
            remaining_days_str = (
                f"{remaining_days[0]} → {remaining_days[-1]} ({len(remaining_days)} днів)"
                if remaining_days else "–"
            )
        else:
            skipped_days_str = "–"
            remaining_days_str = f"{start[:10]} → {end[:10]}"

        _p(f"▶ Починаємо: {total} тікерів (пропущено {len(done_tickers)} по старому маніфесту) | {start} → {end} | interval={interval} | next batch #{batch_idx:04d} | run={run_id}")
        _p(f"   workers: tickers={max_workers_tickers if parallel_tickers else 1}, chunks={max_workers_chunks if parallel_chunks else 0} | blue_ocean={include_blue_ocean} | day-manifest={use_day_manifest}")
        _p(f"   day-manifest: повністю готових тікерів пропущено = {len(fully_done)}")
        _p(f"   📉 пропущені дні (в межах window): {skipped_days_str}")
        _p(f"   📈 обробляються дні: {remaining_days_str}")
        _p(_mem_str(f"   {psutil_note} · старт · "))

    def _make_batch_path(idx: int) -> str:
        fname = f"{out_prefix}_{idx:04d}.csv" + (".gz" if gzip else "")
        return os.path.join(out_dir, fname)

    def _append_df_for_buffer(dfi: pd.DataFrame):
        nonlocal rows_in_buffer
        if dfi is None or dfi.empty:
            return
        buffer.append(dfi)
        rows_in_buffer += len(dfi)

    # NEW: миттєво оновлюємо in-memory кеш днів із нових рядків
    def _update_day_cache_from_df(dfi: pd.DataFrame):
        if not use_day_manifest or dfi is None or dfi.empty:
            return
        if "ticker" not in dfi.columns or "dt" not in dfi.columns:
            return
        tmp = dfi[["ticker", "dt"]].copy()
        tmp["date"] = tmp["dt"].astype(str).str.slice(0, 10)
        for tk, grp in tmp.groupby("ticker"):
            done_days_by_ticker.setdefault(str(tk).upper(), set()).update(grp["date"].astype(str).tolist())

    def _flush():
        nonlocal buffer, batch_idx, written_total, rows_in_buffer, last_flush_at
        if not buffer:
            return None
        path = _make_batch_path(batch_idx)
        while os.path.exists(path):
            batch_idx += 1
            path = _make_batch_path(batch_idx)

        if verbose:
            _p(_mem_str("   ⏳ перед flush · "))

        out = pd.concat(buffer, ignore_index=True)

        # Stable order & dedup
        if "dt" in out.columns:
            out = out.drop_duplicates(subset=["ticker","dt"], keep="last")
            out = out.sort_values(["ticker","dt"], kind="mergesort").reset_index(drop=True)

        tmp_path = path + ".tmp"
        if gzip:
            out.to_csv(tmp_path, index=False, compression={"method": "gzip", "compresslevel": 1})
        else:
            out.to_csv(tmp_path, index=False, compression=None)
        os.replace(tmp_path, path)

        written_total += len(out)
        if verbose:
            _p(f"💾 batch {batch_idx} → {len(out):,} рядків (всього {written_total:,} у цьому запуску) → {path}")

        # NEW: update day-manifest (та одразу оновити кеш у пам'яті)
        if use_day_manifest:
            if "dt" not in out.columns:
                for alt in ("datetime", "time", "t"):
                    if alt in out.columns:
                        out = out.rename(columns={alt: "dt"})
                        break
            _append_done_days_manifest(out_dir, out)
            _update_day_cache_from_df(out)
        else:
            if "ticker" in out.columns:
                _append_done_tickers_manifest(out_dir, out["ticker"].astype(str).unique().tolist())

        # (опційно) відмічати повністю закриті тікери у старому маніфесті, якщо він увімкнений
        if use_manifest and use_day_manifest:
            def _ticker_fully_done_local(tk: str) -> bool:
                s = done_days_by_ticker.get(str(tk).upper(), set())
                return bool(s) and all(dd in s for dd in all_days_str)
            fully_done_now = []
            for tk in out.get("ticker", pd.Series(dtype=str)).astype(str).str.upper().unique():
                if _ticker_fully_done_local(tk):
                    fully_done_now.append(tk)
            _append_done_tickers_manifest(out_dir, fully_done_now)

        batch_idx += 1
        buffer.clear()
        rows_in_buffer = 0
        del out
        gc.collect()
        last_flush_at = time.time()

        if verbose:
            _p(_mem_str("   ✅ після flush · "))
        return path

    def _progress_line(done: int) -> str:
        pct = (done / total * 100) if total else 100.0
        elapsed = time.time() - start_time
        per_item = elapsed / done if done > 0 else 0.0
        remaining = (total - done) * per_item if done > 0 else 0.0
        return f"{pct:6.2f}% | ETA {_fmt_eta(remaining)}"

    def _should_print(idx: int, is_error: bool, is_last: bool) -> bool:
        if is_error or is_last:
            return True
        if progress_every and progress_every > 0:
            return (idx % progress_every == 0)
        return True

    def _maybe_log_mem(idx: int):
        if not verbose:
            return
        if mem_log_every and mem_log_every > 0 and idx % mem_log_every == 0:
            _p(_mem_str("   • RAM · "))

    def _maybe_flush_by_thresholds(processed: int):
        if not buffer:
            return
        need_rows = (flush_when_rows is not None and rows_in_buffer >= flush_when_rows)
        need_time = (flush_every_seconds is not None and (time.time() - last_flush_at) >= flush_every_seconds)
        need_periodic = (save_every and processed % save_every == 0)
        if need_rows or need_time or need_periodic:
            _flush()

    # NEW: якщо за діапазон взагалі не прийшли дані — позначаємо дні як виконані
    def _mark_days_done(tk: str, days_list: List[pd.Timestamp]):
        if not use_day_manifest or not days_list:
            return
        df_mark = pd.DataFrame({
            "ticker": str(tk).upper(),
            "dt": [pd.Timestamp(d).strftime("%Y-%m-%d 00:00:00") for d in days_list],
        })
        _append_done_days_manifest(out_dir, df_mark)
        done_days_by_ticker.setdefault(str(tk).upper(), set()).update(
            [pd.Timestamp(d).strftime("%Y-%m-%d") for d in days_list]
        )

    # --- per-ticker worker (respects day-manifest gaps) ---
    def _one(tk: str):
        try:
            # determine which days to fetch
            all_days = all_days_list
            if use_day_manifest:
                done_set = done_days_by_ticker.get(str(tk).upper(), set())
                todo_days = [d for d in all_days if d.strftime("%Y-%m-%d") not in done_set]
            else:
                todo_days = all_days

            if not todo_days:
                return tk, pd.DataFrame(), None

            # group missing days into ranges (capped by chunk_days)
            todo_ranges = _group_consecutive_dates(todo_days, max_span_days=max(1, int(chunk_days)))

            parts: List[pd.DataFrame] = []
            if parallel_chunks and len(todo_ranges) > 1:
                w = max(1, min(int(max_workers_chunks), len(todo_ranges) * (2 if include_blue_ocean else 1)))
                with ThreadPoolExecutor(max_workers=w) as ex:
                    futs = []
                    for (a, b) in todo_ranges:
                        futs.append(ex.submit(
                            _one_call_v3_dt_fast, tk, a, b, interval,
                            chart_type="ohlcv", include_boats=False, normalize_ohlcv=False
                        ))
                        if include_blue_ocean:
                            futs.append(ex.submit(
                                _one_call_blue_ocean_v3_dt_fast, tk, a, b, interval,
                                chart_type="ohlcv", normalize_ohlcv=False
                            ))
                    for f in as_completed(futs):
                        df = f.result()
                        if df is not None and not df.empty:
                            parts.append(df)
            else:
                for (a, b) in todo_ranges:
                    df_main = _one_call_v3_dt_fast(tk, a, b, interval, chart_type="ohlcv", include_boats=False, normalize_ohlcv=False)
                    if df_main is not None and not df_main.empty:
                        parts.append(df_main)
                    if include_blue_ocean:
                        df_bo = _one_call_blue_ocean_v3_dt_fast(tk, a, b, interval, chart_type="ohlcv", normalize_ohlcv=False)
                        if df_bo is not None and not df_bo.empty:
                            parts.append(df_bo)

            if not parts:
                # не прийшло даних — позначаємо ці дні відпрацьованими, щоб більше не ходити
                _mark_days_done(tk, todo_days)  # NEW
                return tk, pd.DataFrame(), None

            dfo = pd.concat(parts, ignore_index=True)

            if "dt" in dfo.columns:
                dfo = dfo.drop_duplicates(subset=["ticker","dt"], keep="last")
                dfo = dfo.sort_values(["ticker","dt"], kind="mergesort").reset_index(drop=True)

            # завжди залишаємо 'ticker' та 'dt' (щоб маніфест працював)
            if keep_columns:
                base = ["ticker", "dt"]
                cols = base + [c for c in keep_columns if c in dfo.columns and c not in base]
                cols = [c for c in cols if c in dfo.columns]
                dfo = dfo[cols]

            return tk, dfo, None
        except Exception as e:
            return tk, None, str(e)

    if parallel_tickers:
        with ThreadPoolExecutor(max_workers=max_workers_tickers) as ex:
            futs = {ex.submit(_one, tk): tk for tk in tickers}
            processed = 0
            for f in as_completed(futs):
                tk = futs[f]
                processed += 1
                is_last = (processed == total)
                try:
                    tk_, df, err = f.result()
                except Exception as e:
                    errors.append({"ticker": tk, "error": str(e)})
                    if _should_print(processed, True, is_last) and verbose:
                        _p(f"[{processed}/{total}] ✗ {tk}: {e} | {_progress_line(processed)}")
                    _maybe_log_mem(processed)
                    continue

                if err:
                    errors.append({"ticker": tk_, "error": err})
                    if _should_print(processed, True, is_last) and verbose:
                        _p(f"[{processed}/{total}] ✗ {tk_}: {err} | {_progress_line(processed)}")
                else:
                    if df is not None and not df.empty:
                        _append_df_for_buffer(df)
                        _update_day_cache_from_df(df)
                        if _should_print(processed, False, is_last) and verbose:
                            _p(f"[{processed}/{total}] ✓ {tk_}: {len(df):,} рядків | {_progress_line(processed)}")
                    else:
                        if _should_print(processed, False, is_last) and verbose:
                            _p(f"[{processed}/{total}] – {tk_}: empty | {_progress_line(processed)}")

                del df
                gc.collect()
                _maybe_log_mem(processed)
                _maybe_flush_by_thresholds(processed)
    else:
        for i, tk in enumerate(tickers, 1):
            is_last = (i == total)
            tk_, df, err = _one(tk)
            if err:
                errors.append({"ticker": tk_, "error": err})
                if _should_print(i, True, is_last) and verbose:
                    _p(f"[{i}/{total}] ✗ {tk_}: {err} | {_progress_line(i)}")
            else:
                if df is not None and not df.empty:
                    _append_df_for_buffer(df)
                    _update_day_cache_from_df(df)
                    if _should_print(i, False, is_last) and verbose:
                        _p(f"[{i}/{total}] ✓ {tk_}: {len(df):,} рядків | {_progress_line(i)}")
                else:
                    if _should_print(i, False, is_last) and verbose:
                        _p(f"[{i}/{total}] – {tk_}: empty | {_progress_line(i)}")

            del df
            gc.collect()
            _maybe_log_mem(i)
            _maybe_flush_by_thresholds(i)

    _flush()
    total_elapsed = time.time() - start_time
    if verbose:
        _p(_mem_str("   🔚 фінал · "))
        _p(f"🏁 Готово. Нових рядків: {written_total:,} → {out_dir} | Час: {_fmt_eta(total_elapsed)}")
    return pd.DataFrame(errors)


In [17]:
top1500

,ticker,value,lvl3,country,exchange,market_cap
0,A,53276.79,Medical Equipment & Devices,UNITED STATES,New York,38456.0
1,AA,1196849.64,Metals & Mining,UNITED STATES,New York,15162.0
2,AAAU,1291460.13,NaN,NaN,NaN,NaN
3,AACG,36.46,Software,UNITED STATES,NASDAQ CM,34.0
4,AACI,16875.54,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
6693,ZYBT,71.57,Biotech & Pharma,UNITED STATES,NASDAQ CM,48.0
6694,ZYME,49039.53,Biotech & Pharma,UNITED STATES,NASDAQ GS,1727.0
6695,ZYN,412.24,NaN,NaN,NaN,NaN
6696,ZYXI,4482.24,NaN,NaN,NaN,NaN


In [18]:
import pandas as pd

# список тикерів
tickers = ['SPY','IWM','GDX','XLF','QQQ','XLE','KRE','BITO','KWEB','SOXL']

# створення DataFrame
df = pd.DataFrame(tickers, columns=['ticker'])

print(df)


  ticker
0    SPY
1    IWM
2    GDX
3    XLF
4    QQQ
5    XLE
6    KRE
7   BITO
8   KWEB
9   SOXL


In [19]:
# keep = ["ticker","dt","o","h","l","c","v"]

# errs = fetch_intraday_v3_for_all_tickers(
#     tickers_df=top1500,
#     start= start,
#     end= end,
#     interval=1,
#     chunk_days=30,
#     parallel_chunks=True,
#     max_workers_chunks=6,
#     parallel_tickers=True,
#     max_workers_tickers=1,
#     save_every=50,
#     out_dir=str(BATCH_DIR),
#     out_prefix="intraday",
#     gzip=True,
#     keep_columns=keep,
#     verbose=True,
#     include_blue_ocean=True,
# )


In [20]:
import os, glob, gc, time
from typing import Optional

import pandas as pd

try:
    import pyarrow as pa
    import pyarrow.parquet as pq
except Exception:
    pa = pq = None


def collect_premarket_by_day_stream(
    folder: str,
    out_path: str = "CRACEN/work/premarket_rows.parquet",
    pattern: str = "intraday_*.csv.gz",
    start_time: str = "00:00:00",
    end_time: str   = "23:59:00",
    *,
    date_min: Optional[str] = None,     # 'YYYY-MM-DD' inclusive
    date_max: Optional[str] = None,     # 'YYYY-MM-DD' inclusive
    aggregate: bool = False,            # False → всі рядки, True → агрегація по (ticker,date)
    chunksize: int = 200_000,           # якщо мало RAM — ще зменш
    compression: str = "snappy",
    log_every: int = 5,
    sort_chunk: bool = True,            # sort within each chunk (ticker,dt)
):
    """
    Проходить по {folder}/{pattern}, фільтрує рядки:
      - за часовим вікном [start_time..end_time] по підрядку часу в 'dt'
      - опційно за датою [date_min..date_max] по підрядку дати в 'dt'
    і стрімово пише у один Parquet.

    Очікувані колонки у CSV: 'ticker','dt','o','h','l','c','v'.
    dt має формат 'YYYY-MM-DD HH:MM:SS' (string).
    """
    if pq is None or pa is None:
        raise RuntimeError("Потрібен pyarrow: pip install pyarrow")

    files = sorted(glob.glob(os.path.join(folder, pattern)))
    if not files:
        raise FileNotFoundError(f"Не знайдено файлів {pattern} у {folder}")

    # normalize dates (string compare works for YYYY-MM-DD)
    if date_min is not None:
        date_min = str(date_min)[:10]
    if date_max is not None:
        date_max = str(date_max)[:10]
    if date_min is not None and date_max is not None and date_min > date_max:
        date_min, date_max = date_max, date_min

    # читаємо тільки потрібне, даємо вузькі типи
    usecols = ["ticker", "dt", "o", "h", "l", "c", "v"]
    dtypes = {
        "ticker": "string",
        "dt": "string",         # важливо: фільтруємо по підрядку, без datetime-парсингу
        "o": "float32",
        "h": "float32",
        "l": "float32",
        "c": "float32",
        "v": "float32",
    }

    # підготовка виходу
    if os.path.exists(out_path):
        os.remove(out_path)
    writer = None

    t0 = time.time()
    total_in = total_out = 0
    file_idx = 0

    rng_str = ""
    if date_min or date_max:
        rng_str = f"  date=[{date_min or '…'}..{date_max or '…'}]"
    print(f"▶️ START: {folder}/{pattern} → {out_path}  chunksize={chunksize:,}{rng_str}")

    try:
        for path in files:
            file_idx += 1
            chunk_idx = 0

            reader = pd.read_csv(
                path,
                compression="infer",
                usecols=usecols,
                dtype=dtypes,
                chunksize=chunksize,
                low_memory=True,
                # engine="pyarrow",
            )

            for chunk in reader:
                chunk_idx += 1
                total_in += len(chunk)

                # швидке фільтрування по часу: 'YYYY-MM-DD HH:MM:SS'
                t = chunk["dt"].str.slice(11, 19)
                mask = (t >= start_time) & (t <= end_time)

                # фільтр по даті, також строками
                if date_min is not None or date_max is not None:
                    d = chunk["dt"].str.slice(0, 10)
                    if date_min is not None:
                        mask &= (d >= date_min)
                    if date_max is not None:
                        mask &= (d <= date_max)

                cut = chunk.loc[mask].copy()

                if cut.empty:
                    del chunk, t, mask, cut
                    gc.collect()
                    continue

                if aggregate:
                    # дата як рядок 'YYYY-MM-DD'
                    cut["date"] = cut["dt"].str.slice(0, 10)
                    # сорт для стабільності
                    if sort_chunk and {"ticker", "date"}.issubset(cut.columns):
                        cut = cut.sort_values(["ticker", "date"], kind="mergesort")
                    grp = (cut.groupby(["ticker", "date"], as_index=False, observed=True)
                              .agg({"v": "sum"}))  # приклад: сума обсягу; змінюй агрегацію за потреби
                    out_df = grp
                else:
                    # стабільний порядок в межах чанка
                    if sort_chunk and {"ticker", "dt"}.issubset(cut.columns):
                        cut = cut.sort_values(["ticker", "dt"], kind="mergesort")
                    out_df = cut

                table = pa.Table.from_pandas(out_df, preserve_index=False)
                if writer is None:
                    writer = pq.ParquetWriter(out_path, table.schema, compression=compression)
                writer.write_table(table)
                total_out += len(out_df)

                if (chunk_idx % log_every) == 0:
                    elapsed = time.time() - t0
                    print(f"  [{file_idx}/{len(files)}] {os.path.basename(path)} "
                          f"chunk {chunk_idx}  in={total_in:,}  out={total_out:,}  "
                          f"elapsed={elapsed:,.1f}s")

                del chunk, t, mask, cut, out_df, table
                gc.collect()

    finally:
        if writer is not None:
            writer.close()

    print(f"✅ DONE  in={total_in:,} → out={total_out:,}  to {out_path}  in {time.time()-t0:,.1f}s")
    return out_path


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [21]:
collect_premarket_by_day_stream(
    folder=str(BATCH_DIR),
    out_path=str(WORK_DIR / "premarket_rows.parquet"),
    start_time="00:00:00",
    end_time="23:59:00",
    date_min=start_date_str,      # NEW
    date_max=end_date_str,        # NEW
    aggregate=False,
    chunksize=150_000,
)


RuntimeError: Потрібен pyarrow: pip install pyarrow

In [ ]:
import os
import glob
import pandas as pd
from typing import Optional, Dict, Tuple

def build_oc_from_intraday_folder(
    folder: str,
    out_path: Optional[str] = None,
    chunksize: int = 250_000,
) -> pd.DataFrame:
    files = sorted(
        glob.glob(os.path.join(folder, "intraday_*.csv")) +
        glob.glob(os.path.join(folder, "intraday_*.csv.gz"))
    )
    if not files:
        raise FileNotFoundError(f"Не знайдено intraday_*.csv[.gz] у: {folder}")

    OPEN_FROM, OPEN_TO   = "09:30", "09:35"
    CLOSE_FROM, CLOSE_TO = "15:55", "15:59"

    best_open:  Dict[Tuple[str, str], Tuple[str, float]] = {}
    best_close: Dict[Tuple[str, str], Tuple[str, float]] = {}

    usecols = ["ticker", "dt", "o", "c"]

    for path in files:
        for chunk in pd.read_csv(path, compression="infer", low_memory=False,
                                 chunksize=chunksize, usecols=usecols):
            cols = set(map(str, chunk.columns))
            missing = set(usecols) - cols
            if missing:
                raise KeyError(f"{os.path.basename(path)}: відсутні колонки {missing}")

            dt = chunk["dt"].astype(str)
            if "o" in chunk.columns:
                chunk["o"] = pd.to_numeric(chunk["o"], errors="coerce")
            if "c" in chunk.columns:
                chunk["c"] = pd.to_numeric(chunk["c"], errors="coerce")

            hhmm = dt.str.slice(11, 16)
            date = dt.str.slice(0, 10)

            # OPEN
            mask_o = (hhmm >= OPEN_FROM) & (hhmm <= OPEN_TO) & chunk["o"].notna()
            if mask_o.any():
                cut = chunk.loc[mask_o, ["ticker", "o"]].copy()
                cut["date"] = date[mask_o].values
                cut["dt"]   = dt[mask_o].values
                cut = cut.sort_values("dt").drop_duplicates(["ticker","date"], keep="first")
                for r in cut.itertuples(index=False):
                    key = (r.ticker, r.date)
                    cur = best_open.get(key)
                    if (cur is None) or (r.dt < cur[0]):
                        best_open[key] = (r.dt, float(r.o))

            # CLOSE
            mask_c = (hhmm >= CLOSE_FROM) & (hhmm <= CLOSE_TO) & chunk["c"].notna()
            if mask_c.any():
                cut = chunk.loc[mask_c, ["ticker", "c"]].copy()
                cut["date"] = date[mask_c].values
                cut["dt"]   = dt[mask_c].values
                cut = cut.sort_values("dt").drop_duplicates(["ticker","date"], keep="last")
                for r in cut.itertuples(index=False):
                    key = (r.ticker, r.date)
                    cur = best_close.get(key)
                    if (cur is None) or (r.dt > cur[0]):
                        best_close[key] = (r.dt, float(r.c))

    opens_df = pd.DataFrame(
        [(t, d, v) for (t, d), (_, v) in best_open.items()],
        columns=["ticker", "date", "open"]
    )
    closes_df = pd.DataFrame(
        [(t, d, v) for (t, d), (_, v) in best_close.items()],
        columns=["ticker", "date", "close"]
    )

    out = pd.merge(opens_df, closes_df, on=["ticker","date"], how="outer") \
            .sort_values(["ticker","date"], kind="stable") \
            .reset_index(drop=True)

    # ⬇️ ОКРУГЛЕННЯ ДО 2 ЗНАКІВ
    if "open" in out.columns:
        out["open"] = pd.to_numeric(out["open"], errors="coerce").round(2)
    if "close" in out.columns:
        out["close"] = pd.to_numeric(out["close"], errors="coerce").round(2)

    # завжди Parquet
    if out_path is None:
        out_path = os.path.join(folder, "oc.parquet")
    if not out_path.endswith(".parquet"):
        out_path += ".parquet"
    os.makedirs(os.path.dirname(out_path) or ".", exist_ok=True)
    out.to_parquet(out_path, index=False)

    return out


In [ ]:
df_oc = build_oc_from_intraday_folder(
    folder=str(BATCH_DIR),
    out_path=str(WORK_DIR / "oc.parquet"),
    chunksize=250_000
)
print(df_oc.head())


In [ ]:
import os
import gc
import time
import pandas as pd

try:
    import pyarrow as pa
    import pyarrow.parquet as pq
except Exception:
    pq = None


def enrich_with_day_open_and_prev_close(df_intraday: pd.DataFrame,
                                        df_oc: pd.DataFrame) -> pd.DataFrame:
    """
    Додає до кожного рядка інтрадею 'open' дня та 'prev_close' (учорашній close).
    Очікує в df_intraday: ['ticker','dt',...]
              в df_oc:       ['ticker','date','open','close']
    """
    out = df_intraday.copy()
    out["dt"] = pd.to_datetime(out["dt"], errors="coerce")
    out = out[out["dt"].notna()].copy()
    out["date"] = out["dt"].dt.date.astype("string")

    oc = df_oc.copy()
    oc["date"] = pd.to_datetime(oc["date"], errors="coerce").dt.date.astype("string")

    oc_sorted = (
        oc.sort_values(["ticker", "date"])
          .groupby("ticker", as_index=False, group_keys=False)
          .apply(lambda d: d.assign(prev_close=d["close"].shift(1)))
    )

    right = oc_sorted[["ticker", "date", "open", "prev_close"]].copy()
    out = out.merge(right, on=["ticker", "date"], how="left", validate="m:1")
    out = out.sort_values(["ticker", "dt"]).reset_index(drop=True)
    return out


def merge_intraday_and_oc_to_file(
    intraday_parquet_path: str,
    oc_parquet_path: str,
    out_path: str,
    batch_rows: int = 2_000_000,
    log_every_batches: int = 5,
):
    """
    Читає великий Parquet батчами (через pyarrow.parquet.ParquetFile.iter_batches),
    мерджить із денними open/prev_close з oc.parquet і пише у out_path
    (.parquet або .csv/.csv.gz).

    intraday_parquet_path  — шлях до CRACEN/premarket_rows.parquet
    oc_parquet_path        — шлях до CRACEN/oc.parquet
    out_path               — шлях для результату (у тій же папці)
    """
    if pq is None:
        raise RuntimeError("Потрібен pyarrow (pyarrow.parquet).")

    t0 = time.time()

    # 1) Денний OC (у пам'яті)
    df_oc = pd.read_parquet(oc_parquet_path)
    req_oc = {"ticker", "date", "open", "close"}
    missing = req_oc - set(df_oc.columns)
    if missing:
        raise KeyError(f"OC-файл не містить колонок: {sorted(missing)}")

    # 2) Підготовка вихідного файла
    if os.path.exists(out_path):
        os.remove(out_path)
    _, ext = os.path.splitext(out_path.lower())
    if ext not in [".csv", ".gz", ".parquet", ".csv.gz"]:
        raise ValueError("Підтримані формати: .csv, .csv.gz, .parquet")

    parquet_writer = None
    total_rows = 0
    batch_idx = 0

    # 3) Потокове читання parquet
    pf = pq.ParquetFile(intraday_parquet_path)
    cols = ["ticker", "dt", "o", "h", "l", "c", "v"]

    print(f"▶️ START: merge → {out_path} | batch_rows≈{batch_rows:,}")

    for batch in pf.iter_batches(batch_size=batch_rows, columns=cols):
        batch_idx += 1
        df_batch = batch.to_pandas()

        df_enriched = enrich_with_day_open_and_prev_close(df_batch, df_oc)

        if out_path.endswith(".parquet"):
            table = pa.Table.from_pandas(df_enriched)
            if parquet_writer is None:
                parquet_writer = pq.ParquetWriter(out_path, table.schema, compression="snappy")
            parquet_writer.write_table(table)
        else:
            header = not os.path.exists(out_path)
            df_enriched.to_csv(
                out_path,
                index=False,
                mode="a",
                header=header,
                compression="infer" if out_path.endswith(".gz") else None,
            )

        total_rows += len(df_batch)
        if batch_idx % log_every_batches == 0:
            elapsed = time.time() - t0
            print(f"[batch {batch_idx:>4}] rows={total_rows:,}  elapsed={elapsed:,.1f}s")

        del df_batch, df_enriched, batch
        gc.collect()

    if parquet_writer is not None:
        parquet_writer.close()

    print(f"✅ DONE  rows={total_rows:,}  -> {out_path}  in {time.time()-t0:,.1f}s")


def merge_intraday_and_oc_in_folder(
    folder: str = "CRACEN",
    intraday_filename: str = "premarket_rows.parquet",
    oc_filename: str = "oc.parquet",
    out_filename: str = "premarket_rows_enriched.parquet",
    batch_rows: int = 2_000_000,
    log_every_batches: int = 5,
) -> str:
    """
    Зручний обгортник саме під твою папку зі скріну.
    Шукає:
      - {folder}/{intraday_filename}
      - {folder}/{oc_filename}
    і пише:
      - {folder}/{out_filename}
    Повертає шлях до результату.
    """
    intraday_path = os.path.join(folder, intraday_filename)
    oc_path = os.path.join(folder, oc_filename)
    out_path = os.path.join(folder, out_filename)

    if not os.path.exists(intraday_path):
        raise FileNotFoundError(f"Не знайдено інтрадей: {intraday_path}")
    if not os.path.exists(oc_path):
        raise FileNotFoundError(f"Не знайдено OC-файл: {oc_path}")
    os.makedirs(folder, exist_ok=True)

    merge_intraday_and_oc_to_file(
        intraday_parquet_path=intraday_path,
        oc_parquet_path=oc_path,
        out_path=out_path,
        batch_rows=batch_rows,
        log_every_batches=log_every_batches,
    )
    return out_path


In [ ]:
out_path = merge_intraday_and_oc_in_folder(folder=str(WORK_DIR))
print("Результат у файлі:", out_path)


In [ ]:
import os, time, gc
import pandas as pd
import numpy as np
from typing import Optional

try:
    import pyarrow as pa
    import pyarrow.parquet as pq
except Exception:
    pq = None


def add_pct_vs_prev_close_parquet(
    input_path: str,
    output_path: str,
    batch_rows: int = 2_000_000,
    round_to: Optional[int] = None,
    log_every: int = 5,
    value_col: str = "o",        # ← що порівнюємо з prev_close (дефолт: open)
    out_col: str = "Stack%",     # назва колонки з відсотком
) -> str:
    """
    Потоково читає великий Parquet, додає колонку:
        out_col = (value_col / prev_close - 1) * 100
    і пише новий Parquet.
    """
    if pq is None:
        raise RuntimeError("Потрібен pyarrow (pyarrow.parquet).")

    t0 = time.time()
    if os.path.exists(output_path):
        os.remove(output_path)

    pf = pq.ParquetFile(input_path)
    parquet_writer = None
    total = 0
    batches = 0

    print(f"▶️ START  file={input_path}  →  {output_path}  batch_rows≈{batch_rows:,}  using={value_col}")

    for batch in pf.iter_batches(batch_size=batch_rows):
        batches += 1
        df = batch.to_pandas()

        # перевірки на наявність колонок
        if "prev_close" not in df.columns:
            raise KeyError("У вхідному файлі немає колонки 'prev_close'.")
        if value_col not in df.columns:
            raise KeyError(f"У вхідному файлі немає колонки '{value_col}'.")

        # числові типи
        df["prev_close"] = pd.to_numeric(df["prev_close"], errors="coerce")
        df[value_col]    = pd.to_numeric(df[value_col],    errors="coerce")

        # нульовий prev_close → NaN, щоб не ділити на 0
        denom = df["prev_close"].replace({0: np.nan})

        # розрахунок % vs prev_close
        pct = (df[value_col] / denom - 1.0) * 100.0

        # ще раз гарантуємо float dtype (без object/pd.NA всередині)
        pct = pd.to_numeric(pct, errors="coerce")

        if round_to is not None:
            pct = pct.round(int(round_to))

        df[out_col] = pct

        table = pa.Table.from_pandas(df, preserve_index=False)
        if parquet_writer is None:
            parquet_writer = pq.ParquetWriter(
                output_path,
                table.schema,
                compression="snappy",
            )
        parquet_writer.write_table(table)

        total += len(df)
        if batches % log_every == 0:
            print(f"[batch {batches:>4}] rows={total:,}  elapsed={time.time()-t0:,.1f}s")

        del df, batch, table
        gc.collect()

    if parquet_writer is not None:
        parquet_writer.close()

    print(f"✅ DONE  rows={total:,}  →  {output_path}  in {time.time()-t0:,.1f}s")
    return output_path


def add_pct_vs_prev_close_in_folder(
    folder: str = "CRACEN",
    input_filename: str = "premarket_rows_enriched.parquet",
    output_filename: str = "premarket_rows_enriched_with_pct.parquet",
    batch_rows: int = 2_000_000,
    round_to: Optional[int] = 2,
    log_every: int = 5,
    value_col: str = "o",          # ← open vs prev_close
    out_col: str = "Stack%",       # назва нової колонки
) -> str:
    """Обгортка під твою папку CRACEN."""
    os.makedirs(folder, exist_ok=True)
    src = os.path.join(folder, input_filename)
    dst = os.path.join(folder, output_filename)
    if not os.path.exists(src):
        raise FileNotFoundError(f"Не знайдено вхідний файл: {src}")
    return add_pct_vs_prev_close_parquet(
        input_path=src,
        output_path=dst,
        batch_rows=batch_rows,
        round_to=round_to,
        log_every=log_every,
        value_col=value_col,
        out_col=out_col,
    )


In [ ]:
out_path = add_pct_vs_prev_close_in_folder(folder=str(WORK_DIR))
print("Готово:", out_path)


In [ ]:
import os, re, time, math
import pandas as pd
from typing import Dict, List, Optional

try:
    import pyarrow as pa
    import pyarrow.parquet as pq
except Exception:
    pa = pq = None


# ---------- НАДІЙНЕ ЧИТАННЯ МАПІНГУ (ticker, bench) ----------

def _read_mapping(mapping_path: str) -> pd.DataFrame:
    """
    Читає файл з відповідностями тікер↔бенч. Підтримка:
      - CSV (роздільник ; або , або \t), з/без заголовка
      - XLSX
      - Випадок "все в одній колонці": 'AAPL;SPY' / 'AAPL,SPY'
    Повертає DF: ['ticker','bench'] (UPPER, без дублікатів).
    """
    p = mapping_path
    ext = os.path.splitext(p)[1].lower()

    if ext in (".xlsx", ".xls"):
        df = pd.read_excel(p)
    else:
        # 1) спроба autodetect
        try:
            df = pd.read_csv(p, sep=None, engine="python")
        except Exception:
            # 2) спроби з явними роздільниками
            for sep_try in (";", ",", "\t", "|"):
                try:
                    df = pd.read_csv(p, sep=sep_try, engine="python")
                    break
                except Exception:
                    df = None
            if df is None:
                # 3) можливо, файл без заголовка
                try:
                    df = pd.read_csv(p, header=None, sep=None, engine="python")
                except Exception:
                    # 4) остання спроба — явні сепаратори без заголовка
                    for sep_try in (";", ",", "\t", "|"):
                        try:
                            df = pd.read_csv(p, header=None, sep=sep_try, engine="python")
                            break
                        except Exception:
                            df = None
                    if df is None:
                        raise

    # Якщо одна колонка і виглядає як "AAPL;SPY" — сплітимо
    if df.shape[1] == 1:
        col = df.columns[0]
        # намагаємось розбити за ; або , або \t
        s = df[col].astype(str)
        if s.str.contains(";").any():
            parts = s.str.split(";", n=1, expand=True)
        elif s.str.contains(",").any():
            parts = s.str.split(",", n=1, expand=True)
        elif s.str.contains("\t").any():
            parts = s.str.split("\t", n=1, expand=True)
        else:
            raise KeyError("Не можу знайти роздільник між 'ticker' і 'bench'.")
        df = pd.DataFrame({"ticker": parts[0], "bench": parts[1]})

    # Вгадуємо назви колонок
    cols = {c.lower(): c for c in df.columns}
    tcol = cols.get("ticker") or cols.get("symbol")
    bcol = cols.get("bench") or cols.get("benchmark") or cols.get("etf")

    # Якщо заголовків нема, але 2 колонки — приймаємо їх як ticker/bench
    if tcol is None or bcol is None:
        if df.shape[1] >= 2:
            tcol = df.columns[0]
            bcol = df.columns[1]
        else:
            raise KeyError("Потрібні колонки 'ticker' і 'bench' у mapping-файлі.")

    out = (
        df[[tcol, bcol]]
        .rename(columns={tcol: "ticker", bcol: "bench"})
        .dropna()
    )
    out["ticker"] = out["ticker"].astype(str).str.strip().str.upper()
    out["bench"]  = out["bench"].astype(str).str.strip().str.upper()
    out = out[out["ticker"] != ""]
    out = out[out["bench"]  != ""]
    return out.drop_duplicates(ignore_index=True)


# ---------- ВИТЯГ corr/beta ДЛЯ ПАР (X, ФІКСОВАНИЙ BENCH) ----------

def _fetch_corr_beta_pairs(
    x_list: List[str],
    bench: str,
    *,
    period: str = "6 months",
    fmt: str = "json_records",
    batch_size: int = 150,
    min_batch_size: int = 10,
    pause: float = 0.15,
    max_retries: int = 3,
    retry_backoff: float = 0.6,
) -> pd.DataFrame:
    results: List[pd.DataFrame] = []
    i, n = 0, len(x_list)
    b = bench

    while i < n:
        cur_bs = min(batch_size, n - i)
        attempt = 0
        while True:
            xs = x_list[i : i + cur_bs]
            params = {
                "x_tickers": ",".join(xs),
                "y_tickers": b,
                "period": period,
                "as_pivot": False,
                "format": fmt,
            }
            try:
                df_chunk = DatumApi.data_request("/calculations/corr_beta/v2", params)
                if df_chunk is not None and not df_chunk.empty:
                    # лишаємо тільки рядки для потрібного bench (на випадок, якщо API поверне зайве)
                    if "y_ticker" in df_chunk.columns:
                        df_chunk = df_chunk[df_chunk["y_ticker"].astype(str).str.upper() == b]
                    results.append(df_chunk)
                i += len(xs)
                time.sleep(pause)  # пауза тільки після успішного батча
                break
            except Exception as e:
                msg = str(e).lower()
                # 414: зменшуємо ЛИШЕ поточний батч
                if "414" in msg or "uri too large" in msg:
                    if cur_bs <= min_batch_size:
                        print(f"⚠️ 414 навіть при batch={cur_bs}. Пропускаю X[{i}:{i+cur_bs}] для {b}.")
                        i += cur_bs
                        break
                    new_bs = max(min_batch_size, cur_bs // 2)
                    print(f"⚠️ 414 → shrink {cur_bs} → {new_bs} для {b} і повторюю.")
                    cur_bs = new_bs
                    continue

                attempt += 1
                if attempt > max_retries:
                    print(f"⚠️  Пропуск X[{i}:{i+cur_bs}] для {b} після {max_reTRIES} ретраїв. Помилка: {e}")
                    i += cur_bs
                    break
                sleep_s = retry_backoff * (2 ** (attempt - 1))
                print(f"⚠️  {e} → retry {attempt}/{max_retries} через {sleep_s:.1f}s")
                time.sleep(sleep_s)

    return pd.concat(results, ignore_index=True) if results else pd.DataFrame()


# ---------- ГОЛОВНА ФУНКЦІЯ ----------

def corr_beta_from_mapping_to_parquet(
    mapping_path: str,
    out_parquet_path: str,
    *,
    period: str = "3 months",
    batch_size: int = 150,
    min_batch_size: int = 10,
    pause: float = 0.15,
    max_retries: int = 3,
    retry_backoff: float = 0.6,
) -> str:
    """
    Читає mapping (ticker, bench) будь-якого адекватного формату,
    групує тікери по бенчу, витягує corr/beta для кожної групи
    і потоково пише результат у Parquet.
    """
    if pq is None or pa is None:
        raise RuntimeError("Потрібен pyarrow (pa, pq) для запису .parquet")

    map_df = _read_mapping(mapping_path)

    groups: Dict[str, List[str]] = (
        map_df.groupby("bench")["ticker"].apply(lambda s: sorted(set(s.tolist()))).to_dict()
    )

    if os.path.exists(out_parquet_path):
        os.remove(out_parquet_path)

    writer: Optional[pq.ParquetWriter] = None
    total_rows = 0
    t0 = time.time()

    print(f"▶️ START corr/beta → {out_parquet_path} | період={period} | бенчів={len(groups)}")

    for b, xs in sorted(groups.items()):
        if not xs:
            continue
        print(f"— Bench {b}: {len(xs)} тикерів")
        df_b = _fetch_corr_beta_pairs(
            xs, b,
            period=period,
            batch_size=batch_size,
            min_batch_size=min_batch_size,
            pause=pause,
            max_retries=max_retries,
            retry_backoff=retry_backoff,
        )
        if df_b is None or df_b.empty:
            continue

        # типи
        for col in ("corr", "beta"):
            if col in df_b.columns:
                df_b[col] = pd.to_numeric(df_b[col], errors="coerce")

        table = pa.Table.from_pandas(df_b, preserve_index=False)
        if writer is None:
            writer = pq.ParquetWriter(out_parquet_path, table.schema, compression="snappy")
        writer.write_table(table)
        total_rows += len(df_b)

    if writer is not None:
        writer.close()

    print(f"✅ DONE rows={total_rows:,} → {out_parquet_path}  in {time.time()-t0:,.1f}s")
    return out_parquet_path


In [ ]:
out_path = corr_beta_from_mapping_to_parquet(
    mapping_path=str(WORK_DIR / "ticker_bench_.csv"),
    out_parquet_path=str(WORK_DIR / "corr_beta_pairs.parquet"),
    period="3 months",
    batch_size=150,
    pause=0.15,
)
print("Результат:", out_path)


In [ ]:

y_list = ["SPY", "IWM", "GDX", "XLF", "QQQ", "XLE", "KRE", "BITO", "SOXL"]



In [ ]:
import os, time, gc
import pandas as pd
from typing import Optional

try:
    import pyarrow as pa
    import pyarrow.parquet as pq
except Exception:
    pa = pq = None


def _load_corrbeta_mapping_parquet(path: str) -> pd.DataFrame:
    """
    Читає parquet з мапою:
      A) ['x_ticker','y_ticker','corr','beta'] (API-форма), або
      B) ['ticker','bench','corr','beta'] (вже нормалізована).
    Повертає DF: ['ticker','bench','corr','beta'] (UPPER, без дублікатів; при дублях
    лишаємо рядок із найбільшою corr).
    """
    if pq is None:
        raise RuntimeError("Потрібен pyarrow для читання parquet.")

    # Читаємо БЕЗ фільтра колонок — так надійніше для різних схем
    table = pq.read_table(path)
    df = table.to_pandas()

    # нормалізуємо імена колонок для пошуку
    name_map = {c: c.strip().lower() for c in df.columns}
    inv_map = {v: k for k, v in name_map.items()}

    has_api    = ("x_ticker" in inv_map) and ("y_ticker" in inv_map)
    has_direct = ("ticker" in inv_map) and ("bench" in inv_map)

    if not (has_api or has_direct):
        # допоміжна підказка, що саме є в файлі
        available = ", ".join(df.columns.astype(str).tolist())
        raise KeyError(
            "Mapping parquet повинен містити або ['x_ticker','y_ticker','corr','beta'], "
            "або ['ticker','bench','corr','beta'].\n"
            f"У файлі наявні колонки: {available}"
        )

    if has_api:
        df = df.rename(columns={
            inv_map["x_ticker"]: "ticker",
            inv_map["y_ticker"]: "bench"
        })
    else:
        df = df.rename(columns={
            inv_map["ticker"]: "ticker",
            inv_map["bench"]:  "bench"
        })

    # corr/beta можуть бути відсутні — створимо
    if "corr" not in name_map.values():
        df["corr"] = pd.NA
    else:
        df = df.rename(columns={inv_map["corr"]: "corr"})
    if "beta" not in name_map.values():
        df["beta"] = pd.NA
    else:
        df = df.rename(columns={inv_map["beta"]: "beta"})

    # залишаємо тільки потрібне
    keep = [c for c in ["ticker", "bench", "corr", "beta"] if c in df.columns]
    df = df[keep].copy()

    # нормалізація значень
    df["ticker"] = df["ticker"].astype(str).str.strip().str.upper()
    df["bench"]  = df["bench"].astype(str).str.strip().str.upper()
    if "corr" in df.columns:
        df["corr"] = pd.to_numeric(df["corr"], errors="coerce")
    if "beta" in df.columns:
        df["beta"] = pd.to_numeric(df["beta"], errors="coerce")

    # якщо раптом на один тикер кілька рядків — візьмемо з найбільшою corr
    if "corr" in df.columns:
        df = (df.sort_values(["ticker", "corr"], ascending=[True, False])
                .drop_duplicates(subset=["ticker"], keep="first")
                .reset_index(drop=True))
    else:
        df = df.drop_duplicates(subset=["ticker"], keep="first").reset_index(drop=True)

    # гарантуємо повний набір стовпців
    for c in ("corr", "beta"):
        if c not in df.columns:
            df[c] = pd.NA

    return df[["ticker", "bench", "corr", "beta"]]


def attach_bench_corr_beta_parquet(
    input_parquet_path: str,
    corrbeta_parquet_path: str,
    out_parquet_path: str,
    *,
    batch_rows: int = 2_000_000,
    log_every_batches: int = 5,
    round_decimals: Optional[int] = 4,
    insert_before: str = "pct_vs_prev_close",
    compression: str = "snappy",
) -> str:
    """
    Потоково читає великий Parquet (інтрадеї), мерджить 'bench','corr','beta' з іншого
    Parquet (мапа) по 'ticker' і пише новий Parquet.
    """
    if pq is None or pa is None:
        raise RuntimeError("Потрібен pyarrow (pa, pq) для роботи з Parquet.")

    t0 = time.time()
    mapping = _load_corrbeta_mapping_parquet(corrbeta_parquet_path)
    if round_decimals is not None:
        mapping["corr"] = mapping["corr"].round(round_decimals)
        mapping["beta"] = mapping["beta"].round(round_decimals)

    if os.path.exists(out_parquet_path):
        os.remove(out_parquet_path)

    pf = pq.ParquetFile(input_parquet_path)
    writer = None
    total_rows = 0
    batches = 0

    print(f"▶️ START merge → {out_parquet_path}")

    for batch in pf.iter_batches(batch_size=batch_rows):  # читаємо всі колонки
        batches += 1
        df = batch.to_pandas()

        if "ticker" not in df.columns:
            raise KeyError("У вхідному parquet відсутній стовпець 'ticker'.")
        df["ticker"] = df["ticker"].astype(str).str.strip().str.upper()

        merged = df.merge(mapping, on="ticker", how="left", validate="m:1")

        # Переставимо нові колонки за потреби
        if insert_before in merged.columns:
            cols = list(merged.columns)
            for c in ("bench", "corr", "beta"):
                if c in cols: cols.remove(c)
            idx = cols.index(insert_before)
            new_cols = cols[:idx] + ["bench", "corr", "beta"] + cols[idx:]
            merged = merged[new_cols]
        else:
            base = [c for c in merged.columns if c not in ("bench","corr","beta")]
            merged = merged[base + ["bench","corr","beta"]]

        table = pa.Table.from_pandas(merged, preserve_index=False)
        if writer is None:
            writer = pq.ParquetWriter(out_parquet_path, table.schema, compression=compression)
        writer.write_table(table)

        total_rows += len(merged)
        if batches % log_every_batches == 0:
            print(f"[batch {batches:>4}] rows={total_rows:,}  elapsed={time.time()-t0:,.1f}s")

        del df, merged, batch, table
        gc.collect()

    if writer is not None:
        writer.close()

    print(f"✅ DONE rows={total_rows:,} → {out_parquet_path}  in {time.time()-t0:,.1f}s")
    return out_parquet_path


# опціональний аліас під старе ім’я (щоб існуючий код не міняти)
def attach_bench_corr_beta_file(**kwargs) -> str:
    return attach_bench_corr_beta_parquet(**kwargs)


In [ ]:
out_path = attach_bench_corr_beta_parquet(
    input_parquet_path=str(WORK_DIR / "premarket_rows_enriched_with_pct.parquet"),
    corrbeta_parquet_path=str(WORK_DIR / "corr_beta_pairs.parquet"),
    out_parquet_path=str(WORK_DIR / "premarket_rows_final.parquet"),
    batch_rows=2_000_000,
    round_decimals=4,
    insert_before="pct_vs_prev_close",
    compression="snappy",
)
print("Готово:", out_path)


In [ ]:
import os
import pandas as pd

def peek_parquet(path: str, n: int = 20, columns=None):
    """
    Легкий перегляд великого Parquet:
      - друкує шлях, кількість рядків (із метаданих), кількість колонок і їх назви
      - потоково читає і показує перші n рядків (optionally лише вказані columns)
      - повертає DataFrame із прев’ю (n рядків)
    Вимагає pyarrow.
    """
    try:
        import pyarrow.parquet as pq
    except Exception as e:
        raise RuntimeError(
            "Потрібен пакет 'pyarrow' для читання parquet. Встанови: pip install pyarrow"
        ) from e

    if not os.path.exists(path):
        raise FileNotFoundError(path)

    pf = pq.ParquetFile(path)
    total_rows = pf.metadata.num_rows if pf.metadata is not None else None
    schema_cols = [f.name for f in pf.schema_arrow]
    print(f"Файл: {path}")
    if total_rows is not None:
        print(f"Рядків (meta): {total_rows:,}")
    print(f"Колонок: {len(schema_cols)}")
    print("Схема:", ", ".join(schema_cols))

    # Збираємо перші n рядків потоково
    need = n
    parts = []
    for batch in pf.iter_batches(batch_size=min(100_000, max(1, n)), columns=columns):
        df_part = batch.to_pandas()
        parts.append(df_part)
        need -= len(df_part)
        if need <= 0:
            break

    head_df = pd.concat(parts, ignore_index=True).head(n) if parts else pd.DataFrame()
    # Коротка діагностика по ключових нових стовпцях (якщо є)
    for c in ["bench", "corr", "beta", "pct_vs_prev_close"]:
        if c in head_df.columns:
            na = int(head_df[c].isna().sum())
            print(f"NaN у '{c}' (у прев'ю): {na}/{len(head_df)}")

    print("\nПерші рядки:")
    with pd.option_context("display.max_rows", n, "display.max_columns", 200, "display.width", 200):
        print(head_df)

    return head_df


In [ ]:
peek_parquet("CRACEN/work/premarket_rows_final.parquet", n=620)


In [ ]:
# список тикерів
tickers = ['SPY','IWM','GDX','XLF','QQQ','XLE','KRE','BITO','KWEB','SOXL']

# створення DataFrame
df_bench = pd.DataFrame(tickers, columns=['ticker'])


In [ ]:
keep = ["ticker","dt","o","h","l","c","v"]

errs = fetch_intraday_v3_for_all_tickers(
    tickers_df=df_bench,
    start= start,
    end= end,
    interval=1,
    chunk_days=30,
    parallel_chunks=True,
    max_workers_chunks=6,
    parallel_tickers=True,
    max_workers_tickers=1,
    save_every=50,
    out_dir=str(ETF_DIR),
    out_prefix="intraday",
    gzip=True,
    keep_columns=keep,
    verbose=True,
    include_blue_ocean=True,   # ⬅️ додай це
)


In [ ]:
collect_premarket_by_day_stream(
    folder=str(ETF_DIR),
    out_path=str(WORK_DIR / "premarket_etf_rows.parquet"),
    start_time="00:00:00",
    end_time="23:59:00",
    aggregate=False,
    chunksize=150_000,
)


In [ ]:
df_oc = build_oc_from_intraday_folder(
    folder=str(ETF_DIR),
    out_path=str(WORK_DIR / "oc_etf.parquet"),
    chunksize=250_000
)
print(df_oc.head())


In [ ]:
import os
import gc
import time
import pandas as pd

try:
    import pyarrow as pa
    import pyarrow.parquet as pq
except Exception:
    pq = None


def enrich_with_day_open_and_prev_close(df_intraday: pd.DataFrame,
                                        df_oc: pd.DataFrame) -> pd.DataFrame:
    """
    Додає до кожного рядка інтрадею 'open' дня та 'prev_close' (учорашній close).
    Очікує в df_intraday: ['ticker','dt',...]
              в df_oc:       ['ticker','date','open','close']
    """
    out = df_intraday.copy()
    out["dt"] = pd.to_datetime(out["dt"], errors="coerce")
    out = out[out["dt"].notna()].copy()
    out["date"] = out["dt"].dt.date.astype("string")

    oc = df_oc.copy()
    oc["date"] = pd.to_datetime(oc["date"], errors="coerce").dt.date.astype("string")

    oc_sorted = (
        oc.sort_values(["ticker", "date"])
          .groupby("ticker", as_index=False, group_keys=False)
          .apply(lambda d: d.assign(prev_close=d["close"].shift(1)))
    )

    right = oc_sorted[["ticker", "date", "open", "prev_close"]].copy()
    out = out.merge(right, on=["ticker", "date"], how="left", validate="m:1")
    out = out.sort_values(["ticker", "dt"]).reset_index(drop=True)
    return out


def merge_intraday_and_oc_to_file(
    intraday_parquet_path: str,
    oc_parquet_path: str,
    out_path: str,
    batch_rows: int = 2_000_000,
    log_every_batches: int = 5,
):
    """
    Читає великий Parquet батчами (через pyarrow.parquet.ParquetFile.iter_batches),
    мерджить із денними open/prev_close з oc.parquet і пише у out_path
    (.parquet або .csv/.csv.gz).

    intraday_parquet_path  — шлях до CRACEN/premarket_rows.parquet
    oc_parquet_path        — шлях до CRACEN/oc.parquet
    out_path               — шлях для результату (у тій же папці)
    """
    if pq is None:
        raise RuntimeError("Потрібен pyarrow (pyarrow.parquet).")

    t0 = time.time()

    # 1) Денний OC (у пам'яті)
    df_oc = pd.read_parquet(oc_parquet_path)
    req_oc = {"ticker", "date", "open", "close"}
    missing = req_oc - set(df_oc.columns)
    if missing:
        raise KeyError(f"OC-файл не містить колонок: {sorted(missing)}")

    # 2) Підготовка вихідного файла
    if os.path.exists(out_path):
        os.remove(out_path)
    _, ext = os.path.splitext(out_path.lower())
    if ext not in [".csv", ".gz", ".parquet", ".csv.gz"]:
        raise ValueError("Підтримані формати: .csv, .csv.gz, .parquet")

    parquet_writer = None
    total_rows = 0
    batch_idx = 0

    # 3) Потокове читання parquet
    pf = pq.ParquetFile(intraday_parquet_path)
    cols = ["ticker", "dt", "o", "h", "l", "c", "v"]

    print(f"▶️ START: merge → {out_path} | batch_rows≈{batch_rows:,}")

    for batch in pf.iter_batches(batch_size=batch_rows, columns=cols):
        batch_idx += 1
        df_batch = batch.to_pandas()

        df_enriched = enrich_with_day_open_and_prev_close(df_batch, df_oc)

        if out_path.endswith(".parquet"):
            table = pa.Table.from_pandas(df_enriched)
            if parquet_writer is None:
                parquet_writer = pq.ParquetWriter(out_path, table.schema, compression="snappy")
            parquet_writer.write_table(table)
        else:
            header = not os.path.exists(out_path)
            df_enriched.to_csv(
                out_path,
                index=False,
                mode="a",
                header=header,
                compression="infer" if out_path.endswith(".gz") else None,
            )

        total_rows += len(df_batch)
        if batch_idx % log_every_batches == 0:
            elapsed = time.time() - t0
            print(f"[batch {batch_idx:>4}] rows={total_rows:,}  elapsed={elapsed:,.1f}s")

        del df_batch, df_enriched, batch
        gc.collect()

    if parquet_writer is not None:
        parquet_writer.close()

    print(f"✅ DONE  rows={total_rows:,}  -> {out_path}  in {time.time()-t0:,.1f}s")


def merge_intraday_and_oc_in_folder(
    folder: str = "CRACEN",
    intraday_filename: str = "premarket_etf_rows.parquet",
    oc_filename: str = "oc_etf.parquet",
    out_filename: str = "premarket_etf_rows_enriched.parquet",
    batch_rows: int = 2_000_000,
    log_every_batches: int = 5,
) -> str:
    """
    Зручний обгортник саме під твою папку зі скріну.
    Шукає:
      - {folder}/{intraday_filename}
      - {folder}/{oc_filename}
    і пише:
      - {folder}/{out_filename}
    Повертає шлях до результату.
    """
    intraday_path = os.path.join(folder, intraday_filename)
    oc_path = os.path.join(folder, oc_filename)
    out_path = os.path.join(folder, out_filename)

    if not os.path.exists(intraday_path):
        raise FileNotFoundError(f"Не знайдено інтрадей: {intraday_path}")
    if not os.path.exists(oc_path):
        raise FileNotFoundError(f"Не знайдено OC-файл: {oc_path}")
    os.makedirs(folder, exist_ok=True)

    merge_intraday_and_oc_to_file(
        intraday_parquet_path=intraday_path,
        oc_parquet_path=oc_path,
        out_path=out_path,
        batch_rows=batch_rows,
        log_every_batches=log_every_batches,
    )
    return out_path


In [ ]:
out_path = merge_intraday_and_oc_in_folder(folder=str(WORK_DIR))
print("Результат у файлі:", out_path)


In [ ]:
import os, time, gc
import pandas as pd
from typing import Optional

try:
    import pyarrow as pa
    import pyarrow.parquet as pq
except Exception:
    pq = None

def add_pct_vs_prev_close_parquet(
    input_path: str,
    output_path: str,
    batch_rows: int = 2_000_000,
    round_to: Optional[int] = None,
    log_every: int = 5,
    value_col: str = "o",   # ← що порівнюємо з prev_close (дефолт: open)
    out_col: str = "Benchk%",  # назва колонки з відсотком
) -> str:
    """
    Потоково читає великий Parquet, додає колонку:
        out_col = (value_col / prev_close - 1) * 100
    і пише новий Parquet.
    """
    if pq is None:
        raise RuntimeError("Потрібен pyarrow (pyarrow.parquet).")

    t0 = time.time()
    if os.path.exists(output_path):
        os.remove(output_path)

    pf = pq.ParquetFile(input_path)
    parquet_writer = None
    total = 0
    batches = 0

    print(f"▶️ START  file={input_path}  →  {output_path}  batch_rows≈{batch_rows:,}  using={value_col}")

    for batch in pf.iter_batches(batch_size=batch_rows):
        batches += 1
        df = batch.to_pandas()

        # перевірки на наявність колонок
        if "prev_close" not in df.columns:
            raise KeyError("У вхідному файлі немає колонки 'prev_close'.")
        if value_col not in df.columns:
            raise KeyError(f"У вхідному файлі немає колонки '{value_col}'.")

        # числові типи
        df["prev_close"] = pd.to_numeric(df["prev_close"], errors="coerce")
        df[value_col]    = pd.to_numeric(df[value_col],    errors="coerce")

        denom = df["prev_close"].replace({0: pd.NA})
        pct = (df[value_col] / denom - 1.0) * 100.0
        if round_to is not None:
            pct = pct.round(round_to)

        df[out_col] = pct

        table = pa.Table.from_pandas(df, preserve_index=False)
        if parquet_writer is None:
            parquet_writer = pq.ParquetWriter(output_path, table.schema, compression="snappy")
        parquet_writer.write_table(table)

        total += len(df)
        if batches % log_every == 0:
            print(f"[batch {batches:>4}] rows={total:,}  elapsed={time.time()-t0:,.1f}s")

        del df, batch, table
        gc.collect()

    if parquet_writer is not None:
        parquet_writer.close()

    print(f"✅ DONE  rows={total:,}  →  {output_path}  in {time.time()-t0:,.1f}s")
    return output_path


def add_pct_vs_prev_close_in_folder(
    folder: str = "CRACEN",
    input_filename: str = "premarket_etf_rows_enriched.parquet",
    output_filename: str = "premarket_etf_rows_enriched_with_pct.parquet",
    batch_rows: int = 2_000_000,
    round_to: Optional[int] = 2,
    log_every: int = 5,
    value_col: str = "o",                 # ← open vs prev_close
    out_col: str = "Bench%",   # назва нової колонки
) -> str:
    """Обгортка під твою папку CRACEN."""
    os.makedirs(folder, exist_ok=True)
    src = os.path.join(folder, input_filename)
    dst = os.path.join(folder, output_filename)
    if not os.path.exists(src):
        raise FileNotFoundError(f"Не знайдено вхідний файл: {src}")
    return add_pct_vs_prev_close_parquet(
        input_path=src,
        output_path=dst,
        batch_rows=batch_rows,
        round_to=round_to,
        log_every=log_every,
        value_col=value_col,
        out_col=out_col,
    )


In [ ]:
out_path = add_pct_vs_prev_close_in_folder(folder=str(WORK_DIR))
print("Готово:", out_path)


In [ ]:
import os, gc, time
import pandas as pd
from typing import Optional

try:
    import pyarrow as pa
    import pyarrow.parquet as pq
except Exception as e:
    pa = pq = None


def merge_bench_pct_from_etf(
    base_path: str = "CRACEN/work/premarket_rows_enriched_with_pct.parquet",
    etf_path:  str = "CRACEN/work/premarket_etf_rows_enriched_with_pct.parquet",
    out_path:  str = "CRACEN/work/premarket_rows_with_bench_pct.parquet",
    *,
    batch_rows: int = 2_000_000,
    align_to_minute: bool = False,        # якщо в секундах відрізняється — підрівняти до хвилини
    round_decimals: Optional[int] = 2,    # округлити Bench% до 2 знаків
    insert_before: Optional[str] = None,  # наприклад, "pct_vs_prev_close"; інакше в кінець
    compression: str = "snappy",
    log_every_batches: int = 5,
) -> str:
    """
    Мерджить Bench% із ETF-файла у базовий файл по ключах ('bench'↔'ticker', 'dt') і пише новий Parquet.
    - У ETF-файлі колонка з відсотком може називатись 'bench%' або 'pct_vs_prev_close' — обидва варіанти підтримуються.
    - Базовий файл не змінюється, додається тільки нова колонка 'Bench%'.
    """

    if pq is None or pa is None:
        raise RuntimeError("Потрібен pyarrow: pip install pyarrow")

    if not os.path.exists(base_path):
        raise FileNotFoundError(base_path)
    if not os.path.exists(etf_path):
        raise FileNotFoundError(etf_path)

    # ---------- 1) Збираємо мапу з ETF-файла: (bench, dt) -> Bench% ----------
    def _read_etf_mapping(etf_parquet: str) -> pd.DataFrame:
        pf = pq.ParquetFile(etf_parquet)
        parts = []
        # читаємо всі колонки, щоб не ловити різні схеми у row-groups
        for batch in pf.iter_batches():
            df = batch.to_pandas()
            # обов'язкові
            for col in ("ticker", "dt"):
                if col not in df.columns:
                    raise KeyError(f"У ETF-файлі відсутня колонка '{col}'")

            # колонка відсотка: bench% або pct_vs_prev_close
            cols_l = {c.lower().strip(): c for c in df.columns}
            if "bench%" in cols_l:
                val_col = cols_l["bench%"]
            elif "pct_vs_prev_close" in cols_l:
                val_col = cols_l["pct_vs_prev_close"]
            else:
                raise KeyError("У ETF-файлі не знайдено 'bench%' або 'pct_vs_prev_close'.")

            d = df[["ticker", "dt", val_col]].copy()
            d["ticker"] = d["ticker"].astype(str).str.strip().str.upper()
            d["dt"]     = pd.to_datetime(d["dt"], errors="coerce")
            if align_to_minute:
                d["dt"] = d["dt"].dt.floor("T")
            d = d.dropna(subset=["dt"])
            d = d.rename(columns={"ticker": "bench", val_col: "Bench%"})
            d["Bench%"] = pd.to_numeric(d["Bench%"], errors="coerce")
            parts.append(d)

            del df, d, batch
            gc.collect()

        if not parts:
            return pd.DataFrame(columns=["bench","dt","Bench%"])

        etf_map = pd.concat(parts, ignore_index=True)
        # прибираємо дублікати — залишаємо пізніший запис
        etf_map = (etf_map.sort_values(["bench","dt"])
                           .drop_duplicates(subset=["bench","dt"], keep="last")
                           .reset_index(drop=True))
        return etf_map

    etf_map = _read_etf_mapping(etf_path)
    if round_decimals is not None and "Bench%" in etf_map.columns:
        etf_map["Bench%"] = etf_map["Bench%"].round(round_decimals)

    # ---------- 2) Потоково проходимо базовий файл і підмерджуємо Bench% ----------
    if os.path.exists(out_path):
        os.remove(out_path)

    pf_base = pq.ParquetFile(base_path)
    writer = None
    total_rows = 0
    t0 = time.time()

    print(f"▶️ START merge → {out_path}")
    for i, batch in enumerate(pf_base.iter_batches(batch_size=batch_rows), 1):
        df = batch.to_pandas()

        # ключі
        for col in ("bench", "dt"):
            if col not in df.columns:
                raise KeyError(f"У базовому файлі відсутня колонка '{col}'")

        df["bench"] = df["bench"].astype(str).str.strip().str.upper()
        df["dt"]    = pd.to_datetime(df["dt"], errors="coerce")
        if align_to_minute:
            df["dt"] = df["dt"].dt.floor("T")

        merged = df.merge(etf_map, on=["bench","dt"], how="left", validate="m:1")

        # розташування 'Bench%'
        if "Bench%" in merged.columns and insert_before and insert_before in merged.columns:
            cols = list(merged.columns)
            cols.remove("Bench%")
            pos = cols.index(insert_before)
            cols = cols[:pos] + ["Bench%"] + cols[pos:]
            merged = merged[cols]
        # інакше — залишаємо в кінці як є

        table = pa.Table.from_pandas(merged, preserve_index=False)
        if writer is None:
            writer = pq.ParquetWriter(out_path, table.schema, compression=compression)
        writer.write_table(table)

        total_rows += len(merged)
        if i % log_every_batches == 0:
            print(f"[batch {i:>4}] rows={total_rows:,}  elapsed={time.time()-t0:,.1f}s")

        del df, merged, batch, table
        gc.collect()

    if writer is not None:
        writer.close()

    print(f"✅ DONE rows={total_rows:,} → {out_path}  in {time.time()-t0:,.1f}s")
    return out_path


In [ ]:
merge_bench_pct_from_etf(
    base_path=str(WORK_DIR / "premarket_rows_final.parquet"),
    etf_path =str(WORK_DIR / "premarket_etf_rows_enriched_with_pct.parquet"),
    out_path =str(WORK_DIR / "premarket_rows_with_bench_pct.parquet"),
    batch_rows=2_000_000,
    align_to_minute=False,       # вмикай, якщо секунди/мілісекунди «пливуть»
    round_decimals=2,
    insert_before=None,          # або, наприклад, "pct_vs_prev_close"
    compression="snappy",
)


In [ ]:
import pandas as pd
from tqdm import tqdm   # прогрес-бар

def get_gaps_for_tickers(tickers_df, start_date, end_date):
    """
    Отримує гепи для всіх тикерів із DataFrame tickers_df (колонка 'ticker').
    Використовує ендпоінт /daily/gaps і об’єднує всі результати у один DataFrame.
    """
    all_results = []

    tickers = tickers_df["ticker"].dropna().astype(str).unique().tolist()

    for t in tqdm(tickers, desc="Processing tickers"):
        params = {
            "ticker": t,
            "start_date": start_date,
            "end_date": end_date,
            "format": "json_records"
        }

        try:
            df_gap = DatumApi.data_request("/daily/gaps", params)
            if df_gap is not None and not df_gap.empty:
                all_results.append(df_gap)
        except Exception as e:
            print(f"⚠️ Проблема з {t}: {e}")

    if all_results:
        return pd.concat(all_results, ignore_index=True)
    else:
        return pd.DataFrame()

# === приклад використання ===
df_gaps_all = get_gaps_for_tickers(top1500, start_date = start_date_str, end_date = end_date_str)

print(df_gaps_all.head())
print("Загалом рядків:", len(df_gaps_all))


In [ ]:
std_by_ticker = (
    df_gaps_all.groupby("ticker")["gap"]
    .std()                 # std по групі (ddof=1)
    .round(2)              # округлення до 2 знаків
    .reset_index(name="sigma")
)


In [ ]:
std_by_ticker

In [ ]:
import os, time, gc
import pandas as pd
from typing import Optional

try:
    import pyarrow as pa
    import pyarrow.parquet as pq
except Exception:
    pa = pq = None


def _prepare_sigma_df(
    sigma_df: Optional[pd.DataFrame] = None,
    sigma_path: Optional[str] = None
) -> pd.DataFrame:
    """
    Готує мапу ['ticker','sigma'] з DataFrame або з файлу (.parquet/.csv).
    Нормалізує регістр, типи і прибирає дублікати по 'ticker'.
    """
    if sigma_df is None and sigma_path is None:
        raise ValueError("Передай або sigma_df, або sigma_path.")

    if sigma_df is None:
        ext = os.path.splitext(sigma_path)[1].lower()
        if ext == ".parquet":
            sigma_df = pd.read_parquet(sigma_path)
        else:
            sigma_df = pd.read_csv(sigma_path, low_memory=False)

    df = sigma_df.copy()
    # знайдемо колонки гнучко
    cols_lower = {c.lower().strip(): c for c in df.columns}
    tcol = cols_lower.get("ticker")
    scol = cols_lower.get("sigma")
    if tcol is None or scol is None:
        raise KeyError("Очікую колонки 'ticker' і 'sigma' у даних зі скріна.")

    df = df[[tcol, scol]].rename(columns={tcol: "ticker", scol: "sigma"})
    df["ticker"] = df["ticker"].astype(str).str.strip().str.upper()
    df["sigma"]  = pd.to_numeric(df["sigma"], errors="coerce")
    df = (df.sort_values(["ticker"])
            .drop_duplicates(subset=["ticker"], keep="last")
            .reset_index(drop=True))
    return df[["ticker", "sigma"]]


def add_sigma_by_ticker_to_parquet(
    base_path: str = "CRACEN/work/premarket_rows_with_bench_pct.parquet",
    out_path:  Optional[str] = None,
    *,
    sigma_df: Optional[pd.DataFrame] = None,   # напр. твій std_by_ticker зі скріна
    sigma_path: Optional[str] = None,          # або шлях до файла з цими даними
    batch_rows: int = 2_000_000,
    compression: str = "snappy",
    insert_before: Optional[str] = None,       # напр. "Bench%" або "pct_vs_prev_close"; якщо None — в кінець
    log_every_batches: int = 5,
    round_decimals: Optional[int] = 6,
) -> str:
    """
    Потоково додає колонку 'sigma' (за 'ticker') до великого Parquet та зберігає новий файл.
    """
    if pq is None or pa is None:
        raise RuntimeError("Потрібен pyarrow (pyarrow, pyarrow.parquet).")

    if not os.path.exists(base_path):
        raise FileNotFoundError(base_path)

    # вихідний шлях
    if out_path is None:
        folder, fname = os.path.split(base_path)
        name, ext = os.path.splitext(fname)
        out_path = os.path.join(folder, f"{name}_with_sigma{ext or '.parquet'}")

    # підготовка мапи sigma
    sigma_map = _prepare_sigma_df(sigma_df=sigma_df, sigma_path=sigma_path)
    if round_decimals is not None:
        sigma_map["sigma"] = sigma_map["sigma"].round(round_decimals)

    # запис у тимчасовий файл → атомарна заміна
    tmp_path = out_path + f".tmp_{os.getpid()}_{int(time.time())}"
    pf = pq.ParquetFile(base_path)
    writer = None
    total = 0
    t0 = time.time()

    print(f"▶️ START merge sigma → {out_path}")
    for i, batch in enumerate(pf.iter_batches(batch_size=batch_rows), 1):
        df = batch.to_pandas()
        if "ticker" not in df.columns:
            raise KeyError("У базовому файлі відсутня колонка 'ticker'.")

        df["ticker"] = df["ticker"].astype(str).str.strip().str.upper()

        merged = df.merge(sigma_map, on="ticker", how="left", validate="m:1")

        # розташувати 'sigma' за бажанням
        if insert_before and insert_before in merged.columns:
            cols = list(merged.columns)
            cols.remove("sigma")
            idx = cols.index(insert_before)
            cols = cols[:idx] + ["sigma"] + cols[idx:]
            merged = merged[cols]
        else:
            base_cols = [c for c in merged.columns if c != "sigma"]
            merged = merged[base_cols + ["sigma"]]

        table = pa.Table.from_pandas(merged, preserve_index=False)
        if writer is None:
            writer = pq.ParquetWriter(tmp_path, table.schema, compression=compression)
        writer.write_table(table)

        total += len(merged)
        if i % log_every_batches == 0:
            print(f"[batch {i:>4}] rows={total:,}  elapsed={time.time()-t0:,.1f}s")

        del df, merged, batch, table
        gc.collect()

    if writer is not None:
        writer.close()

    # атомарна заміна
    if os.path.exists(out_path):
        os.remove(out_path)
    os.replace(tmp_path, out_path)

    print(f"✅ DONE rows={total:,} → {out_path}  in {time.time()-t0:,.1f}s")
    return out_path


In [ ]:
# std_by_ticker має колонки: ['ticker','sigma']
add_sigma_by_ticker_to_parquet(
    base_path=str(WORK_DIR / "premarket_rows_with_bench_pct.parquet"),
    sigma_df=std_by_ticker,  # <- твій DF зі скріна
    # або sigma_path="CRACEN/std_by_ticker.parquet"/".csv"
    insert_before=None,      # або "Bench%" / "pct_vs_prev_close" — куди вставити
)


In [ ]:
peek_parquet("CRACEN/work/premarket_rows_with_bench_pct_with_sigma.parquet", n=21150)


In [ ]:
import os, time, gc
import pandas as pd
import numpy as np
from typing import Optional

try:
    import pyarrow as pa
    import pyarrow.parquet as pq
    import pyarrow.dataset as ds
except Exception:
    pa = pq = ds = None


def _write_final_sorted_alphabetical(
    input_path: str,
    output_path: str,
    *,
    compression: str = "snappy",
    roundtrip_pandas: bool = True,   # найнадійніше для sort_values
    log_every: int = 200,
):
    """
    Rewrites input parquet into output parquet in strict physical order:
      ticker A→Z, then (date, dt) within each ticker (or dt only).
    Uses dataset scanning per ticker to avoid global in-memory sort.
    """
    if ds is None or pq is None or pa is None:
        raise RuntimeError("Потрібен pyarrow.dataset (pyarrow>=6).")

    dataset = ds.dataset(input_path, format="parquet")

    # Discover columns
    schema_names = [f.name for f in dataset.schema]
    if "ticker" not in schema_names:
        raise KeyError("У вхідному parquet немає колонки 'ticker' — неможливо відсортувати по тикерах.")

    has_date = ("date" in schema_names)
    has_dt   = ("dt" in schema_names)

    # get unique tickers (safe, but could be heavy; acceptable for your use-case)
    tickers_tbl = dataset.to_table(columns=["ticker"])
    tickers = sorted(pd.unique(tickers_tbl.column("ticker").to_pandas().astype(str)))
    del tickers_tbl
    gc.collect()

    tmp_out = output_path + f".tmp_sorted_{os.getpid()}_{int(time.time())}"
    if os.path.exists(tmp_out):
        os.remove(tmp_out)

    writer = None
    total_written = 0
    t0 = time.time()

    print(f"▶ SORT: {input_path} → {output_path} | tickers={len(tickers):,} | has_date={has_date} has_dt={has_dt}")

    try:
        for i, tk in enumerate(tickers, 1):
            # Filter rows for this ticker
            filt = (ds.field("ticker") == tk)
            table = dataset.to_table(filter=filt)
            if table.num_rows == 0:
                continue

            if roundtrip_pandas:
                df = table.to_pandas()
                # Ensure date exists if needed
                if has_date:
                    # keep as-is; assume date is string 'YYYY-MM-DD'
                    pass
                elif has_dt and ("date" not in df.columns):
                    # If no date column exists, optionally derive it (not required for sort)
                    pass

                if has_date and has_dt and {"date", "dt"}.issubset(df.columns):
                    df.sort_values(["date", "dt"], kind="mergesort", inplace=True)
                elif has_dt and "dt" in df.columns:
                    df.sort_values(["dt"], kind="mergesort", inplace=True)
                else:
                    # fallback: stable sort by all columns
                    df.sort_values(list(df.columns), kind="mergesort", inplace=True)

                out_table = pa.Table.from_pandas(df, preserve_index=False)
                del df
            else:
                # Arrow-only fallback (less flexible)
                out_table = table

            if writer is None:
                writer = pq.ParquetWriter(tmp_out, out_table.schema, compression=compression)

            writer.write_table(out_table)
            total_written += out_table.num_rows

            if (i % log_every) == 0:
                print(f"  [{i:>6}/{len(tickers):,}] {tk}  written={total_written:,}  elapsed={time.time()-t0:,.1f}s")

            del table, out_table
            gc.collect()

    finally:
        if writer is not None:
            writer.close()

    # atomic replace
    if os.path.exists(output_path):
        os.remove(output_path)
    os.replace(tmp_out, output_path)

    print(f"✅ SORT DONE: {total_written:,} rows → {output_path} | time={time.time()-t0:,.1f}s")
    return output_path


def recompute_tp_deviation_devsig(
    input_path: str = "CRACEN/work/premarket_rows_with_bench_pct_with_sigma.parquet",
    output_path: str = "CRACEN/work/premarket_rows_with_bench_pct_with_sigma_v2.parquet",
    chunksize: int = 2_000_000,   # використовується як batch_rows для Parquet
    round_tp_dev: Optional[int] = 2,
    round_devsig: Optional[int] = 6,
    compression: str = "snappy",
    *,
    sort_final_by_ticker: bool = True,      # NEW: rewrite final sorted
) -> str:
    """
    Parquet-версія.
    Перерахунок:
      - t_p       = stack% * beta
      - deviation = t_p - bench%
      - dev_sig   = deviation / sigma  (0 -> NaN)
    Стирає старі t_p / deviation / dev_sig, якщо були.
    Працює потоково.

    NEW:
      - якщо sort_final_by_ticker=True, то фінальний output_path буде фізично
        відсортований: ticker A→Z, всередині тикера date,dt (або dt).
    """
    if pq is None or pa is None:
        raise RuntimeError("Потрібен pyarrow (pyarrow, pyarrow.parquet).")

    if not os.path.exists(input_path):
        raise FileNotFoundError(input_path)

    t0 = time.time()
    batch_rows = chunksize

    # 1) write unsorted tmp parquet
    tmp_unsorted = output_path + f".tmp_unsorted_{os.getpid()}_{int(time.time())}"
    if os.path.exists(tmp_unsorted):
        os.remove(tmp_unsorted)

    pf = pq.ParquetFile(input_path)
    writer = None
    total = 0

    print(f"▶ START (Parquet): {input_path}  →  {output_path} | batch_rows≈{batch_rows:,}")

    for bi, batch in enumerate(pf.iter_batches(batch_size=batch_rows), 1):
        df = batch.to_pandas()

        # гнучке визначення колонок
        names = {c.lower().strip(): c for c in df.columns}
        stack_col = names.get("stack%") or names.get("pct_vs_prev_close")
        bench_col = names.get("bench%")
        beta_col  = names.get("beta")
        sigma_col = names.get("sigma")

        need = {
            "stack%/pct_vs_prev_close": stack_col,
            "bench%": bench_col,
            "beta": beta_col,
            "sigma": sigma_col,
        }
        missing = [k for k, v in need.items() if v is None]
        if missing:
            raise KeyError(f"У вхідному файлі бракує колонок: {missing}")

        # приберемо старі результати, якщо були
        for col in ("t_p", "deviation", "dev_sig"):
            if col in df.columns:
                del df[col]

        # числові типи
        df[stack_col] = pd.to_numeric(df[stack_col], errors="coerce")
        df[bench_col] = pd.to_numeric(df[bench_col], errors="coerce")
        df[beta_col]  = pd.to_numeric(df[beta_col],  errors="coerce")
        df[sigma_col] = pd.to_numeric(df[sigma_col], errors="coerce")

        # обчислення
        tp  = df[stack_col] * df[beta_col]
        dev = tp - df[bench_col]
        denom = df[sigma_col].replace({0: np.nan})
        dsg = dev / denom

        # округлення
        if round_tp_dev is not None:
            tp  = tp.round(round_tp_dev)
            dev = dev.round(round_tp_dev)
        if round_devsig is not None:
            dsg = dsg.round(round_devsig)

        # додати в кінець
        df["t_p"] = tp
        df["deviation"] = dev
        df["dev_sig"] = dsg

        table = pa.Table.from_pandas(df, preserve_index=False)
        if writer is None:
            writer = pq.ParquetWriter(tmp_unsorted, table.schema, compression=compression)
        writer.write_table(table)

        total += len(df)
        if bi % 5 == 0:
            print(f"[batch {bi:>4}] rows={total:,}  elapsed={time.time()-t0:,.1f}s")

        del df, batch, table
        gc.collect()

    if writer is not None:
        writer.close()

    # 2) sort & finalize
    if sort_final_by_ticker:
        # rewrite sorted to output_path
        _write_final_sorted_alphabetical(
            tmp_unsorted,
            output_path,
            compression=compression,
            roundtrip_pandas=True,
            log_every=200,
        )
        # cleanup unsorted tmp
        try:
            os.remove(tmp_unsorted)
        except Exception:
            pass
    else:
        # atomic replace unsorted as final
        if os.path.exists(output_path):
            os.remove(output_path)
        os.replace(tmp_unsorted, output_path)

    print(f"✅ DONE: {total:,} rows → {output_path} | total_time={time.time()-t0:,.1f}s")
    return output_path


In [ ]:
recompute_tp_deviation_devsig(
    input_path=str(WORK_DIR / "premarket_rows_with_bench_pct_with_sigma.parquet"),
    output_path=str(FINAL_PATH),
    chunksize=2_000_000,
    round_tp_dev=2,
    round_devsig=6,
)


In [ ]:
import pyarrow.dataset as ds

path = str(FINAL_PATH)

ticker = "AAPL"

dataset = ds.dataset(path, format="parquet")

table = dataset.to_table(
    filter=(ds.field("ticker") == ticker)
)

df = table.to_pandas()

# якщо є date + dt — відсортуємо, щоб це були саме "перші" по часу
if {"date", "dt"}.issubset(df.columns):
    df = df.sort_values(["date", "dt"])

display(df)
